In [0]:
import pandas as pd
import numpy as np
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, LongType, BooleanType, DoubleType, TimestampType, DateType
from pyspark.sql.functions import desc, length, col,isnan,when,count
import pyspark.sql.functions as F
from datetime import datetime, timedelta
from handyspark import *
import re
from unicodedata import normalize

In [0]:
storage_account = "dlsicbfdenuncias"
container = "basecruzada"
blobKey = "LzRpBlOfsiEWD+I6HnUGqQWvLbK4v2IWLc+205FSuhimA8mXWj40UgunJkt2kmMBUAndRTrGuYw2BBM3WVMsIQ=="
blobEndpoint = "wasbs://{1}@{0}.blob.core.windows.net/".format(storage_account, container)

try:
  dbutils.fs.mount(
    source = blobEndpoint,
    mount_point = "/mnt/sample",
    extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})
except:
  print("Already mounted. Unmounting and trying to mount again")
  dbutils.fs.unmount(mount_point='/mnt/sample')
  dbutils.fs.mount(
    source = blobEndpoint,
    mount_point = "/mnt/sample",
    extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})

Already mounted. Unmounting and trying to mount again
/mnt/sample has been unmounted.

In [0]:
# Leer archivo parquet
file = "/mnt/sample/basecruzada.parquet/*.snappy.parquet"
DF_DENUNCIAS = spark.read.parquet(file)

In [0]:
# Ver domensiones y columnas que trae la tabla importada
print((DF_DENUNCIAS.count(), len(DF_DENUNCIAS.columns)))
print(DF_DENUNCIAS.columns)

(1303161, 96)
['NumeroPeticion', 'EsPeticionHija', 'fechaRegistroPeticion', 'AnoRegistroPeticion', 'MesRegistroPeticion', 'TipoPeticion', 'MotivoPeticion', 'RegionalCreacion', 'CZCreacion', 'RegionalDireccionada', 'CZDireccionado', 'CanalRecepcion', 'TipoDocumentoPeticionario', 'NumeroDocPeticionario', 'NombresApellidosPeticionario', 'PrimerNombrePeticionario', 'SegundoNombrePeticionario', 'PrimerApellidoPeticionario', 'SegundoApellidoPeticionario', 'DepartamentoPeticionario', 'MunicipioPeticionario', 'CodDptoPeticionario', 'CodMunicipioPeticionario', 'BarrioVeredaPeticionario', 'DireccionPeticionario', 'SenalesParticularesUbicacionPeticonarioRural', 'DetalleTipoZonaPeticionario', 'DetalleZonaPeticionario', 'EdadPeticionario', 'SexoPeticionario', 'GrupoEtnicoPeticionario', 'PuebloIndigenaPeticionario', 'PresentaDiscapacidadPeticionario', 'CualDiscapacidadPeticionario', 'CondicionDesplazamientoPeticionario', 'HaSidoVictimaPeticionario', 'EstadoPeticion', 'TipoDocumentoAfectado', 'NumeroDocAfectado', 'NombresApellidosAfectado', 'PrimerNombreAfectado', 'SegundoNombreAfectado', 'PrimerApellidoAfectado', 'SegundoApellidoAfectado', 'DepartamentoAfectado', 'MunicipioAfectado', 'CodDptoAfectado', 'CodMunicipioAfectado', 'BarrioVeredaAfectado', 'DireccionAfectado', 'ObsParticUbicaAfectado', 'DetalleTipoZonAfectado', 'DetalleZonAfectado', 'FechaNacAfectado', 'EdadAfectado', 'TipoEdadAfectado', 'SexoAfectado', 'GrupoEtnicoAfectado', 'PuebloIndigenaAfectado', 'PresentaDiscapacidadAfectado', 'CualDiscapacidadAfectado', 'CondicionDesplazamientoAfectado', 'PrimerMotivoIngreso', 'UltimoMotivoIngreso', 'DescripcionPeticion', 'TipoPeticionario', 'CategorizacionPRD_165', 'TipoDemandado', 'D_CONSTATADAS', 'CONSTATACION', 'Val_Constatacion', 'RAVD_Constatadas', 'TIPO_PETICION', 'PRD_160_FECHA_APERTURA', 'ANIO', 'MES', 'UNFICADO_MOTIVO_INGRESO', 'PRD_760_CIERRE_PROCESO_RESTABLECIMIENTO', 'PA/PC', 'FECHA_ACTUACION_MAS_RECIENTE', 'MEDIDA_ACTUACION_MAS_RECIENTE', 'MEDIDA_TOMADA_ULTIMA', 'PRD_165', 'PaisResidenciaPeticionario', 'PaisResidenciaAfectado', 'fecha_ingreso', 'FechaAEG_013', 'FechaPRD_500', 'FechaPRD_510', 'FechaPRD_525', 'FechaPRD_045', 'FechaPRD_825', 'PrimeraFechaPRD_160', 'FechaPRD_165', 'FechaPRD_845', 'PRD_845_AD']

# Iniciar limpieza de base completa

## 1. Verificar campo de ID denominado "NumeroPeticion"

In [0]:
# Verificamos la cantidad de ID nulos
num_ID_nulos = DF_DENUNCIAS.filter(col("NumeroPeticion").isNull())
print("ID NULOS: ", num_ID_nulos.count())

#Verificamos cuántos registros tienen ID duplicados y cuál es el número de ID duplicados
ID_duplicados = DF_DENUNCIAS.groupby("NumeroPeticion").count().filter(col("count")>1).cache()
print("Número de registros con ID duplicados: ", ID_duplicados.agg({'count': 'sum'}).collect()[0][0], 
      "Numero de ID que están duplicados: ", ID_duplicados.count())

# Mostramos los primeros 10 ID más repetidos
print(ID_duplicados.sort("count", ascending=False).show(10))

ID NULOS: 0
Número de registros con ID duplicados: 380 Numero de ID que están duplicados: 190
+--------------+-----+
NumeroPeticion|count|
+--------------+-----+
 19518190| 2|
 16112588| 2|
 17530179| 2|
 26034982| 2|
 26501869| 2|
 29910552| 2|
 14443604| 2|
 11039165| 2|
 12542598| 2|
 16110364| 2|
+--------------+-----+
only showing top 10 rows

None

Para revisar estos duplicados, verificamos si corresponde a registros diferentes (situaciones diferentes, con un mismo ID por errores de la base de datos) o registros duplicados exactos (la información de todos los campos es exactamente el mismo). En el primer caso, se deben eliminar todos los duplicados, ya que no sabemos cuál es la información correcta. En el segundo caso, se debe mantener solo uno de los registros duplicados.

Luego de hacer una verificación uno a uno de los 190 registros duplicados, se encuentra que todos corresponden a NNA que han tenido un PARD, que tuvieron también un reingreso y cuya información no fue registrada correctamente en el sistema de información. Debido a esto, se decide dejar únicamente el registro más reciente, pues es el que tiene la información más actualizada de la medida tomada con el NNA. Para saber cuál es el registro más reciente en cada duplicado, se usarán los campos "ANIO" y "MES" que trae la base de PARD ordenándolos de mayor a menor, y la función dropDuplicates() de Spark dejará solo la primera observación, es decir, la más reciente.

In [0]:
print("Número de registros antes de filtro de ID duplicados: ", DF_DENUNCIAS.count())
DF_DENUNCIAS = DF_DENUNCIAS.sort("NumeroPeticion", "ANIO", "MES").dropDuplicates(subset=["NumeroPeticion"])
print("Número de registros después de filtro de ID duplicados: ", DF_DENUNCIAS.count())

Número de registros antes de filtro de ID duplicados: 1303161
Número de registros después de filtro de ID duplicados: 1302971

## 2. Casteo de variables

In [0]:
# Se crean listas de cómo debe ser casteada cada variable
columnas_longInt = ["NumeroPeticion"]
columnas_int = ['AnoRegistroPeticion',
                'CodDptoPeticionario', 'CodMunicipioPeticionario', 'EdadPeticionario',
                'CodDptoAfectado', 'CodMunicipioAfectado', 'EdadAfectado',
                'ANIO', 'MES']

columnas_float = []

columnas_date = ["fechaRegistroPeticion", "FechaNacAfectado", "PRD_160_FECHA_APERTURA", "PRD_760_CIERRE_PROCESO_RESTABLECIMIENTO",
                 "FECHA_ACTUACION_MAS_RECIENTE", "PRD_165", 'fecha_ingreso', 'FechaAEG_013', 'FechaPRD_500', 'FechaPRD_510', 
                 'FechaPRD_525', 'FechaPRD_045', 'FechaPRD_825', 'PrimeraFechaPRD_160', 'FechaPRD_165', 'FechaPRD_845', 'PRD_845_AD']

# Se define como columnas string todas aquellas que no están en ninguna de las anteriores
columnas_string = [x for x in DF_DENUNCIAS.columns if x not in columnas_int + columnas_float + columnas_date]

In [0]:
# Hay algunas columnas de fecha que traen las fechas en formato decimal de Excel (ej: 44256,8303819444 que representa 1/03/2021  19:55:45). Para ello se debe crear esta funcion convert_date_udf
def convert_date(x):
    mDt = datetime(1899, 12, 30) # Se genera fecha inicial que representa el valor 0 en Excel
    x = str(x)
    if x=="None":
      x = "1900-01-01 00:00:00"
    try:
      x = float(x)
      is_float = True
    except ValueError:
      is_float = False
    if is_float:
      dlt = mDt + timedelta(days=x)
      return dlt.strftime("%Y-%m-%d %H:%M:%S")
    else:
      return x
  
convert_date_udf = udf(lambda z: convert_date(z), StringType())

In [0]:
# Aquí vamos a castear todas las columnas de acuerdo con su tipo
for i in columnas_string:
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, DF_DENUNCIAS[i].cast(StringType()))
for i in columnas_longInt:
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, F.regexp_replace(i, ',', '.').cast(LongType()))
for i in columnas_int:
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, F.regexp_replace(i, ',', '.').cast(IntegerType()))
for i in columnas_float:
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, F.regexp_replace(i, ',', '.').cast(DoubleType()))
for i in columnas_date:
  # Aquí empleamos la función definida en la celda anterior para corregir fechas en formato numérico
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, convert_date_udf(i))
  # Ante posibles errores por variables de fecha, deben incluir el patrón de la fecha acá
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, \
                 when(DF_DENUNCIAS[i].like("_/_/__ __:__"), \
                      F.unix_timestamp(DF_DENUNCIAS[i], "d/M/yy H:mm").cast(TimestampType())). \
                 when(DF_DENUNCIAS[i].like("____-__-__ __:__:__"), \
                      F.unix_timestamp(DF_DENUNCIAS[i], "yyyy-MM-dd HH:mm:ss").cast(TimestampType())). \
                 when(DF_DENUNCIAS[i].like("____-__-__ __:__:__.___"), \
                      F.unix_timestamp(DF_DENUNCIAS[i], "yyyy-MM-dd HH:mm:ss.SSS").cast(TimestampType())). \
                 when(DF_DENUNCIAS[i].like("__/__/____ _:__"), \
                      F.unix_timestamp(DF_DENUNCIAS[i], "dd/MM/yyyy H:mm").cast(TimestampType())). \
                 otherwise( \
                      F.unix_timestamp(DF_DENUNCIAS[i], "dd/MM/yyyy HH:mm").cast(TimestampType())))
  # Fijar que cuando las fechas son del tipo "datetime.datetime(1900, 1, 1, 0, 0) - 1900-01-01 00:00:00" se debe poner como nulo
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, \
                 when(DF_DENUNCIAS[i]==datetime(1900, 1, 1, 0, 0), \
                      None). \
                 otherwise(DF_DENUNCIAS[i]))

In [0]:
# Como pueden ser variables importantes en el modelo, se extrae la hora y minuto de registro de la petición
DF_DENUNCIAS = DF_DENUNCIAS.withColumn("horaRegistroPeticion", F.hour(col("fechaRegistroPeticion")))
DF_DENUNCIAS = DF_DENUNCIAS.withColumn("minutoRegistroPeticion", F.minute(col("fechaRegistroPeticion")))

# Se convierte la variables de TimeStamp a Date. Así se eliminan las referencias a horas, minutos y segundos que tengan las variables y se ponen en un formato estándar para Spark
for i in columnas_date:
  DF_DENUNCIAS = DF_DENUNCIAS.withColumn(i, F.to_date(DF_DENUNCIAS[i]))

## 3. Limpieza de valores vacíos general para todas las variables

In [0]:
# Reemplazar todos los posibles valores nulos
def limpiar_nulos(df, debug=False):
    for contador, i in enumerate(list(df.columns)):
        if debug:
            print(contador, i)
        df = df.withColumn(i, when((col(i) == 'None') | \
            (col(i) == 'NULL') | \
            (col(i) == 'null') | \
            (col(i) == 'NaN') | \
            (F.trim(col(i)) == '') | \
            (col(i) == ' ' ) | \
            (col(i) == '#N/A' ) | \
            (col(i) == 'Sin Información' ) | \
            (col(i) == 'Sin Dato' ) | \
            (col(i) == '#¡REF!' ) | \
            (col(i) == 'VALOR NULO' ) | \
            (col(i) == 'SELECCIONE' ),
            None).otherwise(df[i]))
    return df
  
DF_DENUNCIAS_limpiovacios = limpiar_nulos(DF_DENUNCIAS, debug=False)

# 4. Creación de variable objetivo

A continuación se creara la variable objetivo del modelo con base a los campos "TipoPeticion", "CONSTATACION", "CategorizacionPRD_165" y "MEDIDA_TOMADA_ULTIMA". Las categorías de la variable objetivo serán: falsa, sindefinir_fallida, verdadera_nopard, verdadera_pard_noinst, verdadera_pard_inst

In [0]:
# Frecuencias conjuntas "TipoPeticion" , "CategorizacionPRD_165", "CONSTATACION"
DF_DENUNCIAS_limpiovacios.groupby("TipoPeticion", "CONSTATACION", "CategorizacionPRD_165").count().sort(["TipoPeticion", "CONSTATACION", "CategorizacionPRD_165"]).show(200, truncate=False)

+-----------------------------------------------+-----------------------------+-----------------------------------------------------------+------+
TipoPeticion |CONSTATACION |CategorizacionPRD_165 |count |
+-----------------------------------------------+-----------------------------+-----------------------------------------------------------+------+
Inobservancia de derechos |null |null |7341 |
Inobservancia de derechos |null |ACTIVACION DE RUTA SNBF |491 |
Inobservancia de derechos |null |ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA|61 |
Inobservancia de derechos |null |CONFIRMACIÓN MAYORÍA DE EDAD |1 |
Inobservancia de derechos |null |DATOS DE UBICACION ERRADOS |100 |
Inobservancia de derechos |null |DERECHOS GARANTIZADOS |393 |
Inobservancia de derechos |null |EVASION DEL NIÑO, NIÑA O ADOLESCENTE |13 |
Inobservancia de derechos |null |FALLECIMIENTO DEL NIÑO, NIÑA O ADOLESCENTE |7 |
Inobservancia de derechos |null |PETICIONARIO NO APORTA DATOS DE UBICACION O DE CONTACTO |36 |
Reporte de Amenaza o Vulneración de derechos |null |null |57 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |null |90716 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |ACTIVACION DE RUTA SNBF |21 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA|19 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |DATOS DE UBICACION ERRADOS |565 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |DERECHOS GARANTIZADOS |87 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |EVASION DEL NIÑO, NIÑA O ADOLESCENTE |14 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |PETICIONARIO NO APORTA DATOS DE UBICACION O DE CONTACTO |103 |
Reporte de Amenaza o Vulneración de derechos |Fallida |null |1897 |
Reporte de Amenaza o Vulneración de derechos |Fallida |ACTIVACION DE RUTA SNBF |13 |
Reporte de Amenaza o Vulneración de derechos |Fallida |ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA|16 |
Reporte de Amenaza o Vulneración de derechos |Fallida |DATOS DE UBICACION ERRADOS |77 |
Reporte de Amenaza o Vulneración de derechos |Fallida |DERECHOS GARANTIZADOS |75 |
Reporte de Amenaza o Vulneración de derechos |Fallida |EVASION DEL NIÑO, NIÑA O ADOLESCENTE |9 |
Reporte de Amenaza o Vulneración de derechos |Fallida |FALLECIMIENTO DEL NIÑO, NIÑA O ADOLESCENTE |1 |
Reporte de Amenaza o Vulneración de derechos |Fallida |PETICIONARIO NO APORTA DATOS DE UBICACION O DE CONTACTO |30 |
Reporte de Amenaza o Vulneración de derechos |Falsa |null |141761|
Reporte de Amenaza o Vulneración de derechos |Falsa |ACTIVACION DE RUTA SNBF |132 |
Reporte de Amenaza o Vulneración de derechos |Falsa |ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA|165 |
Reporte de Amenaza o Vulneración de derechos |Falsa |DATOS DE UBICACION ERRADOS |158 |
Reporte de Amenaza o Vulneración de derechos |Falsa |DERECHOS GARANTIZADOS |1417 |
Reporte de Amenaza o Vulneración de derechos |Falsa |EVASION DEL NIÑO, NIÑA O ADOLESCENTE |14 |
Reporte de Amenaza o Vulneración de derechos |Falsa |FALLECIMIENTO DEL NIÑO, NIÑA O ADOLESCENTE |3 |
Reporte de Amenaza o Vulneración de derechos |Falsa |PETICIONARIO NO APORTA DATOS DE UBICACION O DE CONTACTO |50 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |null |63914 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |ACTIVACION DE RUTA SNBF |1753 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA|516 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |CONFIRMACIÓN MAYORÍA DE EDAD |1 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |DATOS DE UBICACION ERRADOS |115 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |DERECHOS GARANTIZADOS |1005 |
Reporte de Amenaza o Vulneración de derechos |Remitido

In [0]:
# Se eliminan todos los registros que corresponden a Inobservancia de Derechos, es decir todos aquellos que en el campo de "TipoPetición" son "Inobservancia de derechos" o en el campo "CategorizacionPRD_165" son "ACTIVACION DE RUTA SNBF"
antes = DF_DENUNCIAS_limpiovacios.count()
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.filter(col("TipoPeticion")!="Inobservancia de derechos")
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.filter((col("CategorizacionPRD_165").isNull()) | (col("CategorizacionPRD_165")!="ACTIVACION DE RUTA SNBF"))
despues = DF_DENUNCIAS_limpiovacios.count()
print("# de registros iniciales: ", antes, "# de registros eliminados: ", antes - despues, "# de registros que permanecen: ", despues)



# de registros iniciales: 1302971 # de registros eliminados: 78796 # de registros que permanecen: 1224175

In [0]:
# Se encuentra que hay discrepancia entre los campos "CategorizacionPRD_165" y "CONSTATACION". Por ejemplo, en unos pocos casos en CONSTATACION aparece Fallida o Falsa, sin embargo en PRD_165 aparece como si se hubiera hecho verificación "ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA". Por esta razón, en la construcción de la variable objetivo se dará prioridad al campo "CategorizacionPRD_165"

# Se crean lsitados de categorías que servirán para la clasificación en la próxima celda
PARD_INST = ["Adopción", "Adopción ", "Atenc especializ internado", "Ub At Especializada Casa Hogar ", "Ub M Fliar Hogar de paso", "Ub M Fliar Hogar Sustituto", "Ubicación en Centro de Emergencia", "Ub M fliar – Solidaridad Familiar", "Ub M Fliar Hogar Amigo"]
PARD_NOINST = ["Acciones policivas", "Acciones policivas, administrativas o judiciales ", "Amonestación con asistencia obligatoria a curso pedagógico. ", "Atenc especiali Intervenciòn de Apoyo", "Atenc especializ Externado", "Atenc especializ semiinternado", "Cualquiera otra medida que garantice la protección integral de los niños, niñas y adolescentes", "Por Definir", "Ub M fliar de origen o familia extensa", "Ub M Fliar Hogar Gestor"]

prd165_srd_pardinst = ["FALLECIMIENTO DEL NIÑO, NIÑA O ADOLESCENTE"]
prd165_srd_verdaderanopard = ["ATENCION POR MEDIO DE ORIENTACION Y ASISTENCIA A LA FAMILIA", "EVASION DEL NIÑO, NIÑA O ADOLESCENTE"]
prd165_srd_sindefinir = ["DATOS DE UBICACION ERRADOS", "CONFIRMACIÓN MAYORÍA DE EDAD", "PETICIONARIO NO APORTA DATOS DE UBICACION O DE CONTACTO"]
prd165_srd_falsa = ["DERECHOS GARANTIZADOS"]

constatacion_verdadera = ["Verdadera"]
constatacion_fallida = ["Dirección Errada", "Fallida", "Sin Resultado de Constatación"]
constatacion_falsa = ["Falsa"]

In [0]:
# Se crean las distintas categorías de la variable objetivo

DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("VAR_OBJETIVO", \
              # Inicialmente definimos la petición verdadera PARD con medida institucional como aquella que cruzó con la base de PARD y en el campo MEDIDA_TOMADA_ULTIMA tiene registrada alguna de las categorías de la lista PARD_INST
              when((col("MEDIDA_TOMADA_ULTIMA").isin(PARD_INST)), \
                     "verdadera_pard_inst"). \
              # También tomaremos como PARD con medida institucional aquellos que en el campo CategorizacionPRD_165 se registran como "FALLECIMIENTO DEL NIÑO, NIÑA O ADOLESCENTE" debido a la gravedad de esta situación que amerita una priorización
              when((col("CategorizacionPRD_165").isin(prd165_srd_pardinst)), \
                     "verdadera_pard_inst"). \
              # Ahora los PARD con medida no institucional son aquellos que están en la lista PARD_NOINST
              when((col("MEDIDA_TOMADA_ULTIMA").isin(PARD_NOINST)), \
                     "verdadera_pard_noinst"). \
              # Ahora las peticiones verdaderas. Tomamos en primera medida todas aquellas (sin importar TipoPeticion) que en el campo CategorizacionPRD_165 estén en las siguientes dos categorías
              when((col("CategorizacionPRD_165").isin(prd165_srd_verdaderanopard)), \
                     "verdadera_nopard"). \
              # También se toman como verdaderas no PARD todas aquellas SRD que no tienen cruce con PARD y CategorizacionPRD_165 es un valor nulo
              when((col("TipoPeticion") == "Solicitud de Restablecimiento de Derechos (SRD)") & \
                   (col("MEDIDA_TOMADA_ULTIMA").isNull()) & \
                   (col("CategorizacionPRD_165").isNull()), \
                     "verdadera_nopard"). \
              # También se toman verdaderas no PARD aquellas RAVD que no cruzan con PARD y tienen resultado de CONSTATACION Verdadera
              when((col("TipoPeticion") == "Reporte de Amenaza o Vulneración de derechos") & \
                   (col("CONSTATACION").isin(constatacion_verdadera)) &\
                   (col("MEDIDA_TOMADA_ULTIMA").isNull()), \
                     "verdadera_nopard"). \
              # Ahora se categorizan las sindefinir_fallida. Estas en primera medida son las que en el campo CategorizacionPRD_165 tiene alguno de los valores de la lista prd165_srd_sindefinir
              when((col("CategorizacionPRD_165").isin(prd165_srd_sindefinir)), \
                     "sindefinir_fallida"). \
              # También aquellas RAVD que tienen en CONSTATACION alguna de las categorías de constatacion_fallida
              when((col("TipoPeticion") == "Reporte de Amenaza o Vulneración de derechos") & \
                   (col("CONSTATACION").isin(constatacion_fallida)) &\
                   (col("MEDIDA_TOMADA_ULTIMA").isNull()), \
                     "sindefinir_fallida"). \
              # Finalmente, las falsas son aquellas que en CategorizacionPRD_165 registra "DERECHOS GARANTIZADOS" o que son RAVD que en el campo CONSTATACION son "Falsa"
              when((col("CategorizacionPRD_165") == "DERECHOS GARANTIZADOS"), \
                     "falsa"). \
              when((col("TipoPeticion") == "Reporte de Amenaza o Vulneración de derechos") & \
                   (col("CONSTATACION") == "Falsa"), \
                     "falsa"). \
            otherwise(None))

In [0]:
# Verificamos frecuencias
DF_DENUNCIAS_limpiovacios.groupby("VAR_OBJETIVO").count().sort("count", ascending=False).show(truncate=False)

+---------------------+------+
VAR_OBJETIVO |count |
+---------------------+------+
falsa |371493|
verdadera_nopard |348201|
sindefinir_fallida |204284|
verdadera_pard_noinst|166827|
verdadera_pard_inst |72438 |
null |60932 |
+---------------------+------+

In [0]:
# Verificamos que los valores nulos corresponden por Remisión a otras entidades
DF_DENUNCIAS_limpiovacios.filter(col("VAR_OBJETIVO").isNull()). \
groupby("VAR_OBJETIVO", "TipoPeticion", "CONSTATACION", "MEDIDA_TOMADA_ULTIMA").count().sort("count", ascending=False).show(truncate=False)

+------------+--------------------------------------------+-----------------------+--------------------+-----+
VAR_OBJETIVO|TipoPeticion |CONSTATACION |MEDIDA_TOMADA_ULTIMA|count|
+------------+--------------------------------------------+-----------------------+--------------------+-----+
null |Reporte de Amenaza o Vulneración de derechos|Remitido a Otra Entidad|null |60910|
null |Reporte de Amenaza o Vulneración de derechos|null |null |22 |
+------------+--------------------------------------------+-----------------------+--------------------+-----+

In [0]:
# Se eliminan todas aquellos registros que tienen VAR_OBJETIVO vacío. Estas corresponden en su totalidad a RAVD que fueron remitidas a otra entidad o de las que no se conoce su resultado de constatación
DF_DENUNCIAS_limpiovacios.cache()
antes = DF_DENUNCIAS_limpiovacios.count()
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.filter(col("VAR_OBJETIVO").isNotNull())
despues = DF_DENUNCIAS_limpiovacios.count()
print("# registros eliminados: ", antes - despues, "# de registros que permanecen: ", despues)

# registros eliminados: 60932 # de registros que permanecen: 1163243

# 5. Creación de variables de valoración PARD
A continuación se crearán tres variables que son de interés para los defensores de familia al momento de consultar el tablero: Si el NNA ha sido atendido o no por un equipo de defensoría de familia, la fecha de esta primera atención y si la petición ha tenido o no un trámite de fondo.

In [0]:
# Las siguientes columnas son las que se van a verificar para saber si el afectado en la petición ha sido atendido por un equipo de defensoría
cols_atendido = ['fecha_ingreso', 'FechaAEG_013', 'FechaPRD_500', 'FechaPRD_510', 'FechaPRD_525', 'FechaPRD_045', 'FechaPRD_825', 'PrimeraFechaPRD_160', 'FechaPRD_165', 'FechaPRD_845', 'PRD_845_AD']

# Se crea columna con primera fecha de atención. Aquellas que no tengan ninguna atención tendrán este campo vacío (null)
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("Fecha_Primera_Atencion", F.least(*cols_atendido))

# Se crea columna llamada "Atendido", en la que si la fecha de primera atención es vacío toma el valor de "NO", pero si tiene algún registro tiene el valor de "SI"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("Atendido",
                                                                 when(col("Fecha_Primera_Atencion").isNull(), "NO").
                                                                 otherwise("SI"))

# Ahora se crea la columna "TramiteDeFondo" para saber si la petición ha tenido un trámite de fondo (en cuyo caso la columna tomará los valores de "PARD", "NoAmeritaPARD" o "RemitidoOtraEntidad") o si no ha tenido trámite de fondo (en este caso tomará el valor de "SinTramite")
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("TramiteDeFondo",
                                                                 when(col("PrimeraFechaPRD_160").isNotNull(), "PARD").
                                                                 when(col("FechaPRD_165").isNotNull(), "NoAmeritaPARD").
                                                                 when(col("FechaPRD_845").isNotNull(), "RemitidoOtraEntidad").
                                                                 otherwise("SinTramite"))

# 6. Creación de variable de PaisAfectado
Se revisará la variable de PaisResidenciaAfectado para verificar su contenido. Además, para el propósito del modelamiento y la visualización se crearán dos variables: Una con los cinco países más frecuentes y el restante agrupado en "Otros" y otra con los diez países más frecuentes y el restante agrupado en "Otros"

In [0]:
# Reemplazar los países que tienen valor vacío (null) por ND
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill("ND", subset="PaisResidenciaAfectado")

# Crear tabla con los 10 países más frecuentes de residencia del afectado, ordenada por frecuencia de mayor a menor
freq_paises = DF_DENUNCIAS_limpiovacios.groupby("PaisResidenciaAfectado").count().sort("count", ascending=False).limit(10)
print(freq_paises.show())

# Convertir el anterior dataframe a una lista que contenga a los 10 primeros países
list_paises_10 = [row['PaisResidenciaAfectado'] for row in freq_paises.collect()]

# Convertir el anterior dataframe a una lista que contenga a los 5 primeros países
list_paises_5 = list_paises_10[:5]

# Crear primera columna en la que se agrupan los países del puesto 11 hacia abajo en "Otros"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("PaisAfectado_10",
                                                                 when(col("PaisResidenciaAfectado").isin(list_paises_10), col("PaisResidenciaAfectado")).
                                                                 otherwise("OTRO"))

# Crear primera columna en la que se agrupan los países del puesto 6 hacia abajo en "Otros"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("PaisAfectado_5",
                                                                 when(col("PaisResidenciaAfectado").isin(list_paises_5), col("PaisResidenciaAfectado")).
                                                                 otherwise("OTRO"))

+----------------------+-------+
PaisResidenciaAfectado| count|
+----------------------+-------+
 COLOMBIA|1125094|
 ND| 34883|
 ESPAÑA| 1006|
 VENEZUELA| 875|
 ARMENIA| 203|
 LIBANO| 165|
 ECUADOR| 110|
 ARGENTINA| 100|
 PERU| 70|
 PUERTO RICO| 61|
+----------------------+-------+

None

# 7. Se agregan variables numéricas adicionales (coordenadas de centro zonal direccionado, variables departamentales)

### 7.1. Obtener coordenadas de centros zonales

In [0]:
#Información sobre conexión a la base de datos.
jdbcHostname = dbutils.secrets.get('blobsecret','SQLHostname')
jdbcDatabase = dbutils.secrets.get('blobsecret','SQLDatabase')
jdbcPort = dbutils.secrets.get('blobsecret','SQLport')
jdbcuser = dbutils.secrets.get('blobsecret','SQLusername')
jdbcpass = dbutils.secrets.get('blobsecret','SQLpassword')
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

# Se lee archivo con coordenadas de centros zonales
coord_cz = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].[centro_zonal_2020]").option("user", jdbcuser).option("password", jdbcpass).load()

# Seleccionamos columnas necesarias
coord_cz = coord_cz.select("CodigoMuni", "COD_CZ", "REGIONAL", "NOMBRE_CEN", "X", "Y")

# Se crea una columna con el codigo DIVIPOLA del departamento y eliminamos la de municipio que no será necesaria
coord_cz = coord_cz.withColumn("CodDptoRegionalDireccionada", F.substring('CodigoMuni', 1,2)).drop("CodigoMuni")

# Se debe arreglar un registro que no tiene 
coord_cz = coord_cz.withColumn("CodDptoRegionalDireccionada", when(col("REGIONAL")=="LA GUAJIRA", "44").otherwise(col("CodDptoRegionalDireccionada")))

# Castear columnas diferentes a texto
for i in ["X", "Y"]:
  coord_cz = coord_cz.withColumn(i, F.regexp_replace(i, ',', '.').cast(DoubleType()))

In [0]:
# Para recuperar las coordenadas de los centros zonales, utilizaremos dos llaves: La regional y el centro zonal. Por lo tanto empezaremos al comparar los campos de regional entre la base de peticiones y el listado de centros zonales que contiene las coordenadas
  
# Valores únicos de regionales a las que se direccionan las peticiones
listaRegional_cz_direccionado = DF_DENUNCIAS_limpiovacios.select("RegionalDireccionada").distinct().sort("RegionalDireccionada", ascending=True).rdd.flatMap(lambda x: x).collect()

# Valores únicos de regionales en el archivo de centros zonales
listaRegional_cz = coord_cz.select("REGIONAL").distinct().sort("REGIONAL", ascending=True).rdd.flatMap(lambda x: x).collect()

# Mostrar cuáles regionales están en base de peticiones pero no en el archivo de centros zonales
print("Regionales que están en la base de peticiones pero no en archivo de centros zonales \n", 
      [x for x in listaRegional_cz_direccionado if x not in listaRegional_cz])

Regionales que están en la base de peticiones pero no en archivo de centros zonales 
 ['INSTITUCIONES AUTORIZADAS ADOPCION', 'NARIÑO', 'SEDE NACIONAL']

In [0]:
# De la celda anterior se ve que es necesario reemplazar en el archivo de centros zonales 'NARIÃ‘O' por 'NARIÑO'. Para ello, se realiza un diccionario y se hace el reemplazo en la variable "REGIONAL"
coord_cz = coord_cz.na.replace({"NARIÃ‘O": "NARIÑO"}, subset="REGIONAL")

# Luego de este cambio, verificar de nuevo cuáles regionales están en base de peticiones pero no en el archivo de centros zonales
listaRegional_cz = coord_cz.select("REGIONAL").distinct().sort("REGIONAL", ascending=True).rdd.flatMap(lambda x: x).collect()
print([x for x in listaRegional_cz_direccionado if x not in listaRegional_cz])

['INSTITUCIONES AUTORIZADAS ADOPCION', 'SEDE NACIONAL']

In [0]:
# Para los centros zonales que tienen como regional INSTITUCIONES AUTORIZADAS ADOPCION' o 'SEDE NACIONAL' será necesario agregar las coordenadas manualmente. Verifiquemos si hay otros registros que tampoco cruzarán y por lo tanto será necesario encontrar las coordenadas de igual forma

# Creamos una función que nos imprimirá el listado de regionales y cz de la base de peticiones que no cruza con las coordenadas

# Primero, creamos una función para limpiar las columnas y facilitar el cruce 
@udf
def limpiar_col(text):
  text = text.lower().replace('.', '').replace('"', '').replace('ã‘', 'ñ').replace(' ', '')
  text = normalize('NFKD', text).encode("ascii","ignore").decode("ascii")
  return text
  
def check_cruce_cz():
  # Seleccionamos columnas necesarias
  check_crucecz_peticiones = DF_DENUNCIAS_limpiovacios.select("RegionalDireccionada", "CZDireccionado").distinct()
  check_crucecz_listado = coord_cz.select("REGIONAL", "NOMBRE_CEN").distinct()

  # Se aplica función a las columnas de regional y centro zonal
  check_crucecz_peticiones = check_crucecz_peticiones.withColumn("RegionalDireccionada2", limpiar_col("RegionalDireccionada"))
  check_crucecz_peticiones = check_crucecz_peticiones.withColumn("CZDireccionado2", limpiar_col("CZDireccionado"))
  check_crucecz_listado = check_crucecz_listado.withColumn("REGIONAL2", limpiar_col("REGIONAL"))
  check_crucecz_listado = check_crucecz_listado.withColumn("NOMBRE_CEN2", limpiar_col("NOMBRE_CEN"))

  # Se realiza el cruce preliminar para conocer cuáles cz no tienen coordenadas. Se hace un filtro de forma que solo queden los que no cruzaron
  cond = [check_crucecz_peticiones["RegionalDireccionada2"] == check_crucecz_listado["REGIONAL2"], 
          check_crucecz_peticiones["CZDireccionado2"] == check_crucecz_listado["NOMBRE_CEN2"]]
  check_crucecz_peticiones = check_crucecz_peticiones.join(check_crucecz_listado, cond, how='left').filter(col("REGIONAL").isNull())

  # Mostrar los registros que no cruzaron
  nocruzaron = check_crucecz_peticiones.select("RegionalDireccionada", "CZDireccionado").sort(["RegionalDireccionada", "CZDireccionado"])
  print("Número de centros zonales en base de peticiones que no cruzaron :", nocruzaron.count())
  print(nocruzaron.show(100, truncate=False))

# Aplicamos la función
check_cruce_cz()

Número de centros zonales en base de peticiones que no cruzaron : 56
+----------------------------------+----------------------------------------+
RegionalDireccionada |CZDireccionado |
+----------------------------------+----------------------------------------+
AMAZONAS |REGIONAL AMAZONAS |
ANTIOQUIA |CAIF COMUNA 13 |
ANTIOQUIA |CAIF COMUNA 8 |
ANTIOQUIA |CZ INTEGRAL NOROCCIDENTAL |
ANTIOQUIA |CZ INTEGRAL NORORIENTAL |
ANTIOQUIA |REGIONAL ANTIOQUIA |
ARAUCA |REGIONAL ARAUCA |
ATLANTICO |REGIONAL ATLANTICO |
BOGOTA |CZ SAN CRISTOBAL SUR |
BOGOTA |REGIONAL BOGOTA |
BOLIVAR |CZ EL CARMEN DE BOLIVAR |
BOLIVAR |REGIONAL BOLIVAR |
BOYACA |REGIONAL BOYACA |
CALDAS |REGIONAL CALDAS |
CAQUETA |REGIONAL CAQUETA |
CASANARE |REGIONAL CASANARE |
CAUCA |REGIONAL CAUCA |
CESAR |CZ AGUSTIN CODAZZI |
CESAR |REGIONAL CESAR |
CHOCO |REGIONAL CHOCO |
CORDOBA |CZ 1 MONTERIA |
CORDOBA |REGIONAL CORDOBA |
CUNDINAMARCA |REGIONAL CUNDINAMARCA |
GUAINIA |REGIONAL GUAINIA |
GUAVIARE |CZ SAN JOSE DE GUAVIARE |
GUAVIARE |REGIONAL GUAVIARE |
HUILA |REGIONAL HUILA |
INSTITUCIONES AUTORIZADAS ADOPCION|"ASOCIACION AMIGOS DEL NIÑO ""AYUDAME"""|
INSTITUCIONES AUTORIZADAS ADOPCION|"FUNDACION ""CRAN""" |
INSTITUCIONES AUTORIZADAS ADOPCION|CASA DE LA MADRE Y EL NIÑO |
INSTITUCIONES AUTORIZADAS ADOPCION|CENTRO DE ADOPCION CHIQUITINES |
INSTITUCIONES AUTORIZADAS ADOPCION|CORPORACION CASA DE MARIA Y EL NIÑO |
INSTITUCIONES AUTORIZADAS ADOPCION|FANA |
INSTITUCIONES AUTORIZADAS ADOPCION|FUNDACION CASITA DE NICOLAS |
INSTITUCIONES AUTORIZADAS ADOPCION|FUNDAQCION LOS PISINGOS |
LA GUAJIRA |REGIONAL LA GUAJIRA |
MAGDALENA |CZ SANTA MARTA 1 |
MAGDALENA |CZ SANTA MARTA 2 |
MAGDALENA |REGIONAL MAGDALENA |
META |REGIONAL META |
NARIÑO |REGIONAL NARIÑO |
NORTE DE SANTANDER |REGIONAL NORTE DE SANTANDER |
PUTUMAYO |REGIONAL PUTUMAYO |
QUINDIO |REGIONAL QUINDIO |
RISARALDA |REGIONAL RISARALDA |
SAN ANDRES |REGIONAL SAN ANDRES |
SANTANDER |CZ YARIQUIES |
SANTANDER |REGIONAL SANTANDER |
SEDE NACIONAL |CENTRO DE CONTACTO |
SEDE NACIONAL |SEDE NACIONAL |
SUCRE |REGIONAL SUCRE |
SUCRE |UNIDAD MÓVIL 01 REGIONAL SUCRE |
TOLIMA |REGIONAL TOLIMA |
VALLE DEL CAUCA |REGIONAL VALLE |
VAUPES |REGIONAL VAUPES |
VICHADA |REGIONAL VICHADA |
+----------------------------------+----------------------------------------+

None

In [0]:
# Se encuentra que la mayoría de regionales y centros zonales que no cruzan son aquellas que en la base de peticiones se direccionan a las direcciones regionales y a las instituciones autorizadas para la adopción. Los restantes son centros zonales que tienen un nombre diferente entre la base de peticiones y la de centros zonales. Estos ajustes será necesario realizarlos manualmente

# Ajustemos primero los centros zonales que tienen un nombre diferente entre las dos bases. Para ello creamos un diccionario que hará los cambios en la base de peticiones
dicc_czcambios = {"CZ EL CARMEN DE BOLIVAR": "CZ CARMEN DE BOLIVAR", "CZ AGUSTIN CODAZZI": "CZ CODAZZI", "CZ 1 MONTERIA": "CZ MONTERIA 1", "CZ INTEGRAL NOROCCIDENTAL": "CZ NOROCCIDENTAL", "CZ INTEGRAL NORORIENTAL": "CZ NORORIENTAL", "CAIF COMUNA 13": "CZ ROSALES", "CZ SAN CRISTOBAL SUR": "CZ SAN CRISTOBAL", "CZ SAN JOSE DE GUAVIARE": "CZ SAN JOSE DEL GUAVIARE", "CZ SANTA MARTA 2": "CZ SANTA MARTA NORTE", "CZ SANTA MARTA 1": "CZ SANTA MARTA SUR", "CAIF COMUNA 8": "CZ SUR ORIENTE", "CZ YARIQUIES": "CZ YARIGUIES"}

DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.replace(dicc_czcambios, subset="CZDireccionado")

# También eliminamos de la columna de centros zonales unos registros que traen comillas "
@udf
def limpiar_col2(text):
  text = text.replace('"', '')
  return text

DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("CZDireccionado", limpiar_col2("CZDireccionado"))

In [0]:
# Se agregan 44 registros con las coordenadas de direcciones regionales e instituciones autorizadas para la adopción
data_cz = [("0000","AMAZONAS","REGIONAL AMAZONAS",-4.2172145,-69.9351763,"91"), ("0000","BOGOTA","REGIONAL BOGOTA",4.6414275,-74.094736,"11"), ("0000","CUNDINAMARCA","REGIONAL CUNDINAMARCA",4.6800036,-74.0625967,"25"), ("0000","CHOCO","REGIONAL CHOCO",5.690288,-76.6606877,"27"), ("0000","ANTIOQUIA","REGIONAL ANTIOQUIA",6.2513882,-75.600688,"05"), ("0000","ARAUCA","REGIONAL ARAUCA",7.0761773,-70.7126457,"81"), ("0000","ATLANTICO","REGIONAL ATLANTICO",10.9903413,-74.7987913,"08"), ("0000","BOLIVAR","REGIONAL BOLIVAR",10.4230524,-75.5471385,"13"), ("0000","BOYACA","REGIONAL BOYACA",5.5762939,-73.3408689,"15"), ("0000","CALDAS","REGIONAL CALDAS",5.0658144,-75.5083546,"17"), ("0000","CAQUETA","REGIONAL CAQUETA",1.6174462,-75.6071531,"18"), ("0000","CASANARE","REGIONAL CASANARE",5.3370571,-72.4087433,"85"), ("0000","CAUCA","REGIONAL CAUCA",2.4468445,-76.6234119,"19"), ("0000","CESAR","REGIONAL CESAR",10.4727743,-73.2500034,"20"), ("0000","CORDOBA","REGIONAL CORDOBA",8.7385536,-75.9056023,"23"), ("0000","GUAINIA","REGIONAL GUAINIA",3.8645172,-67.9270858,"94"), ("0000","GUAVIARE","REGIONAL GUAVIARE",2.5691093,-72.6430071,"95"), ("0000","HUILA","REGIONAL HUILA",2.9328805,-75.3002634,"41"), ("0000","LA GUAJIRA","REGIONAL LA GUAJIRA",11.5363201,-72.9195399,"44"), ("0000","MAGDALENA","REGIONAL MAGDALENA",11.2369312,-74.2063184,"47"), ("0000","META","REGIONAL META",4.1150854,-73.6106049,"50"), ("0000","NARIÑO","REGIONAL NARIÑO",1.2036969,-77.2654141,"52"), ("0000","NORTE DE SANTANDER","REGIONAL NORTE DE SANTANDER",7.9047431,-72.4877553,"54"), ("0000","PUTUMAYO","REGIONAL PUTUMAYO",1.1526395,-76.6512828,"86"), ("0000","QUINDIO","REGIONAL QUINDIO",4.5439624,-75.6703903,"63"), ("0000","RISARALDA","REGIONAL RISARALDA",4.8148492,-75.7101355,"66"), ("0000","SAN ANDRES","REGIONAL SAN ANDRES",12.5617118,-81.7241771,"88"), ("0000","SANTANDER","REGIONAL SANTANDER",7.1437337,-73.1357805,"68"), ("0000","SUCRE","REGIONAL SUCRE",9.2931345,-75.4055886,"70"), ("0000","TOLIMA","REGIONAL TOLIMA",4.4349783,-75.2137268,"73"), ("0000","VALLE DEL CAUCA","REGIONAL VALLE",3.4685525,-76.5190228,"76"), ("0000","VAUPES","REGIONAL VAUPES",1.2595745,-70.2360806,"97"), ("0000","VICHADA","REGIONAL VICHADA",6.1826067,-67.4880416,"99"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","ASOCIACION AMIGOS DEL NIÑO AYUDAME",4.6979682,-74.0772445,"11"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","FUNDACION CRAN",4.7491542,-74.0751404,"11"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","CASA DE LA MADRE Y EL NIÑO",4.6382348,-74.0803545,"11"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","CENTRO DE ADOPCION CHIQUITINES",3.3357044,-76.529711,"76"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","CORPORACION CASA DE MARIA Y EL NIÑO",6.1905439,-75.5659312,"05"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","FANA",4.7535145,-74.0854299,"11"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","FUNDACION CASITA DE NICOLAS",6.2619263,-75.563012,"05"), ("0000","INSTITUCIONES AUTORIZADAS ADOPCION","FUNDAQCION LOS PISINGOS",4.7336526,-74.0267606,"11"), ("0000","SEDE NACIONAL","CENTRO DE CONTACTO",4.6718971,-74.095443,"11"), ("0000","SEDE NACIONAL","SEDE NACIONAL",4.6718971,-74.095443,"11"), ("0000","SUCRE","UNIDAD MÓVIL 01 REGIONAL SUCRE",9.2931345,-75.4055886,"70")]

schema_cz = StructType([ \
    StructField("COD_CZ",StringType(),True), \
    StructField("REGIONAL",StringType(),True), \
    StructField("NOMBRE_CEN",StringType(),True), \
    StructField("X",DoubleType(),True), \
    StructField("Y",DoubleType(),True), \
    StructField("CodDptoRegionalDireccionada",StringType(),True), \
  ])
nuevos_cz = spark.createDataFrame(data=data_cz,schema=schema_cz)

# Unir estos datos al dataframe original de centros zonales
coord_cz = coord_cz.union(nuevos_cz)

In [0]:
# Volvemos a verificar que ahora si todos los centros zonales hayan cruzado
check_cruce_cz()

# Ahora que no hay ningun registro que no cruce, hacemos el cruce final y definitivo para recuperar las coordenadas y el código de centro zonal
# Creamos columnas con nombres limpios con los cuales se hará el cruce
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("RegionalDireccionada2", limpiar_col("RegionalDireccionada"))
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("CZDireccionado2", limpiar_col("CZDireccionado"))
coord_cz = coord_cz.withColumn("REGIONAL2", limpiar_col("REGIONAL"))
coord_cz = coord_cz.withColumn("NOMBRE_CEN2", limpiar_col("NOMBRE_CEN"))

# Se realiza el cruce preliminar para conocer cuáles cz no tienen coordenadas. Se hace un filtro de forma que solo queden los que no cruzaron
cond = [DF_DENUNCIAS_limpiovacios["RegionalDireccionada2"] == coord_cz["REGIONAL2"],
        DF_DENUNCIAS_limpiovacios["CZDireccionado2"] == coord_cz["NOMBRE_CEN2"]]

DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.join(coord_cz, cond, how='left')

# Eliminamos columnas que ya no son necesarias
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.drop("REGIONAL", "REGIONAL2", "NOMBRE_CEN", "NOMBRE_CEN2", "RegionalDireccionada2", "CZDireccionado2")

Número de centros zonales en base de peticiones que no cruzaron : 0
+--------------------+--------------+
RegionalDireccionada|CZDireccionado|
+--------------------+--------------+
+--------------------+--------------+

None

### 7.2. Recuperación de variables descriptivas del nivel departamental
Aquí se empleará un archivo externo que trae variables socioeconómicas a nivel departamental, de forma que se puedan incluir en el modelamiento algunas características propias del departamento adonde es direccionada la petición. El archivo que se importará trae variables como Tasa de Violencia Interpersonal, Tasa de Violencia Intrafamiliar, Tasa de Fecundidad en mujeres entre 15 y 19 años, Porcentaje de población rural, Porcentaje de Población Étnica, Porcentaje de Población Indígena y Víctimas de delitos sexuales. Para esto importaremos el archivo que contiene esta información desde el Storage Account

In [0]:
# Montar de nuevo almacenamiento en contenedor que contiene el archivo con variables de contexto departamental
storage_account = "dlsicbfdenuncias"
container = "data"
blobKey = "LzRpBlOfsiEWD+I6HnUGqQWvLbK4v2IWLc+205FSuhimA8mXWj40UgunJkt2kmMBUAndRTrGuYw2BBM3WVMsIQ=="
blobEndpoint = "wasbs://{1}@{0}.blob.core.windows.net/".format(storage_account, container)

try:
  dbutils.fs.mount(
    source = blobEndpoint,
    mount_point = "/mnt/data",
    extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})
except:
  print("Already mounted. Unmounting and trying to mount again")
  dbutils.fs.unmount(mount_point='/mnt/data')
  dbutils.fs.mount(
    source = blobEndpoint,
    mount_point = "/mnt/data",
    extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account):blobKey})

# Leer archivo como pandas Dataframe
import os
os.chdir(r'/dbfs//mnt/data/Data/')
file = "Variables_contexto_dptal.xlsx"
vardpto_contexto = pd.read_excel(file, sheet_name='importar', engine="openpyxl", converters={'Divipola_dpto1':str,'Divipola_dpto2':str})

# Por la anormalidad de la situación y las cifras del año 2020, para dicho año se tomarán los mismos valores observados en 2019
vars_contexto = ['tasa_vip_total', 'tasa_vif_total', "TasaFecunidad_15a19", "porc_rural", "vict_delsexual_total"]
for i in vars_contexto:
  nom_var_2019, nom_var_2020 = [i + "_2019", i + "_2020"]
  vardpto_contexto[nom_var_2020] = vardpto_contexto[nom_var_2019]

# Convertir el dataframe de formato wide a long
vardpto_contexto = pd.wide_to_long(vardpto_contexto, vars_contexto, i=['Divipola_dpto1', 'Divipola_dpto2', "porc_etnica_2018", "porc_indigena_2018"], j='agno_contexto', sep='_').reset_index()

#Convertir el archivo a Spark Dataframe
vardpto_contexto_sk = spark.createDataFrame(vardpto_contexto)

Already mounted. Unmounting and trying to mount again
/mnt/data has been unmounted.

In [0]:
# Se hace el join o merge con la base de peticiones usando como llaves el código de departamento y que el año de la información de contexto sea del año anterior al de la peticion

#Creamos la columna de rezago del año de contexto
vardpto_contexto_sk = vardpto_contexto_sk.withColumn("agno_contexto_mas1", col("agno_contexto")+1)

# Se realiza el join usando la siguiente condición
cond = [DF_DENUNCIAS_limpiovacios["CodDptoRegionalDireccionada"] == vardpto_contexto_sk["Divipola_dpto2"],
        DF_DENUNCIAS_limpiovacios["AnoRegistroPeticion"] == vardpto_contexto_sk["agno_contexto_mas1"]]

DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.join(vardpto_contexto_sk, cond, how='left')

# Se eliminan columnas innecesarias
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.drop("Divipola_dpto1", "Divipola_dpto2", "agno_contexto", "agno_contexto_mas1")

## 5. Limpieza final de todas las variables, a partir de los resultados anteriores
### 5.1. Empecemos con la variables de fecha "fechaRegistroPeticion", "FechaNacAfectado"

In [0]:
# Ver las fechas más frecuentes para saber cuáles podrían ser valores atípicos o nulos
DF_DENUNCIAS_limpiovacios.groupby("fechaRegistroPeticion").count().sort(desc("count")).show(20)

+---------------------+-----+
fechaRegistroPeticion|count|
+---------------------+-----+
 2018-06-12| 1062|
 2018-02-20| 1053|
 2018-09-25| 1050|
 2018-03-06| 1048|
 2018-05-07| 1046|
 2018-03-20| 1043|
 2018-09-04| 1039|
 2018-06-13| 1037|
 2018-08-21| 1037|
 2018-02-26| 1035|
 2018-07-23| 1019|
 2018-04-25| 1013|
 2018-03-07| 1008|
 2018-02-19| 1008|
 2018-03-14| 1001|
 2018-11-13| 1000|
 2018-08-28| 998|
 2018-09-11| 998|
 2018-08-29| 998|
 2018-09-03| 997|
+---------------------+-----+
only showing top 20 rows

In [0]:
DF_DENUNCIAS_limpiovacios.groupby("FechaNacAfectado").count().sort(desc("count")).show(20)

+----------------+------+
FechaNacAfectado| count|
+----------------+------+
 null|436304|
 2005-01-01| 1725|
 2004-01-01| 1449|
 2003-01-01| 1418|
 2010-01-01| 1346|
 2006-01-01| 1309|
 2002-01-01| 1188|
 2008-01-01| 1164|
 2007-01-01| 1113|
 2001-01-01| 979|
 2009-01-01| 946|
 2000-01-01| 942|
 2015-01-01| 835|
 2011-01-01| 826|
 2012-01-01| 746|
 2013-01-01| 732|
 2014-01-01| 725|
 2016-01-01| 600|
 2017-01-01| 503|
 2018-01-01| 453|
+----------------+------+
only showing top 20 rows

Al parecer las variables de "fechaRegistroPeticion" y "FechaNacAfectado" no tienen  valores atípicos

### 5.2. MotivoPeticion

In [0]:
# Se define diccionario con categorías originales como llaves (keys) y homologaciones/limpieza como valores (values)
dicc_MotivoPeticion = {
    "Amenazado por grupos armados organizados al margen de la ley": "Desvinculados", 
    "Hechos Victimizantes en el Marco del Conflicto Armado": "Desvinculados",
    "Explotación Sexual Comercial \x13†ESCNNA": "ESCNNA", 
    "Pornografía Infantil - Violencia sexual - Explotación Sexual": "ESCNNA", 
    "Prostitución Infantil - Violencia sexual - Explotación Sexual": "ESCNNA",
    "Explotación Sexual Comercial \x96 ESCNNA" : "ESCNNA",
    "Maltrato a Niñ@ en Gestación": "Maltrato", "Maltrato Físico": "Maltrato", 
    "Maltrato físico": "Maltrato", "Maltrato por negligencia": "Maltrato", "Maltrato por Negligencia": "Maltrato", 
    "Maltrato psicológico": "Maltrato", "Maltrato Psicológico": "Maltrato", 
    "Violencia física, psicológica y/o negligencia": "Maltrato",
    "Abandono": "Abandono_Amenaza", "Amenazado": "Abandono_Amenaza",
    "Víctima de uso, porte, manipulación o lesión por pólvora": "Abandono_Amenaza", "Desnutrición": "Abandono_Amenaza",
    "Apoyo a madre gestante o lactante (menor o mayor de 18 años)": "PI_Vulnerable", 
    "Atención a madre gestante Sentencia C-355 de 2006": "PI_Vulnerable", 
    "Atención a niños y niñas hasta los tres (3) años de edad en establecimientos de reclusión de mujeres": "PI_Vulnerable",
    "Carece de representante legal": "NNA_solo", "Carece de Representante Legal": "NNA_solo", "Expósito": "NNA_solo", 
    "Extraviado": "NNA_solo", "Niños, niñas y adolescentes migrantes no acompañados": "NNA_solo","Falta de Responsables": "NNA_solo",
    "Conductas Sexualizadas entre menores de 14 años": "Conductas_Adolescentes", 
    "Conductas sexualizadas entre menores de 14 años": "Conductas_Adolescentes",
    "Consumo de sustancias psicoactivas": "Conductas_Adolescentes", "Consumo de Sustancias Psicoactivas": "Conductas_Adolescentes", 
    "Problemas de comportamiento": "Conductas_Adolescentes", "Problemas de Comportamiento": "Conductas_Adolescentes", 
    "Convivencia Educativa": "Conductas_Adolescentes",
    "Incumplimiento al régimen de visitas y custodia": "Otro", "Motivo SIUCE": "Otro", 
    "NNA Víctimas de desastres naturales u otras situaciones de emergencia": "Otro", 
    "Otorga consentimiento para la adopción": "Otro", "Por establecer": "Otro", "Solicitud vinculación programa de Discapacidad": "Otro",
    "Tenencia Irregular de Niño, Niña o Adolescente": "Otro",
    "Situación de Alta Permanencia en Calle": "Permanencia_Calle",  "Situación de Vida en calle": "Permanencia_Calle", 
    "Situación de vida en calle": "Permanencia_Calle", "Solicitud de refugio de niños, niñas y Adolescentes": "Permanencia_Calle", 
    "None": "ND", 
    "Situación de trabajo infantil": "TI", "Situación Trabajo Infantil (Sin frecuencia y tiempo determinado)": "TI", 
    "Trata con fines diferentes a la explotación sexual": "Trata", "Trata con fines sexuales": "Trata", 
    "Trata de Niños, Niñas y Adolescentes": "Trata", 
    "Abuso Sexual/Violencia Sexual": "V_sexual", "Violencia Sexual": "V_sexual"
}

In [0]:
# Se verifica que las llaves tomadas en el diccionario contengan todas las categorías de la variable
# Para esto creamos una función que nos va a servir para todas las homologaciones/limpieza
def check_replace_llaves(df, var, diccionario):
    keys = list(diccionario.keys()) # Se genera una lista con las llaves del diccionario creado
    df_cuenta_valores = df.select(var).distinct().collect() # Se obtienn los valores únicos de la variable
    lista_valores = [str(row[var]) for row in df_cuenta_valores] # Se realiza una una lista con los valores únicos
    check = set(lista_valores).issubset(keys) # Check es True si las categorías de la variable están contenidas en el diccionario que creamos
    if check==True:
        df = df.na.replace(diccionario, subset=var) # Se usa el diccionario para hacer los reemplazos
        print(check, "= Todas las categorías de la variable {} están incluidas en el diccionario. \n \
        Se hizo el reemplazo de los valores de acuerdo al diccionario.".format(var))
    else:
        print(check, "No se hizo el reemplazo de los valores. \n \
        Aún falta incluir algunas categorías de la variable {} en el diccionario".format(var))
        print("Estas variables son: ", [x for x in lista_valores if x not in keys])
    return df

In [0]:
# Ahora utilizamos la función creada para verificar que el diccionario contenga todas las categorías de la variable
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "MotivoPeticion", diccionario=dicc_MotivoPeticion
)
# Reemplazamos los valores nulos por categoría "Sin información"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill(value="ND",subset=["MotivoPeticion"])
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("MotivoPeticion").count().sort(desc("count")).show()

True = Todas las categorías de la variable MotivoPeticion están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+--------------------+------+
 MotivoPeticion| count|
+--------------------+------+
 Maltrato|594220|
 V_sexual|207041|
Conductas_Adolesc...|164712|
 Otro| 42773|
 TI| 34206|
 Abandono_Amenaza| 32291|
 NNA_solo| 25517|
 Permanencia_Calle| 22818|
 ND| 17750|
 ESCNNA| 9173|
 PI_Vulnerable| 8330|
 Desvinculados| 3428|
 Trata| 984|
+--------------------+------+

## 5.3. Variables de Tipo de documento "TipoDocumentoPeticionario" y "TipoDocumentoAfectado"

In [0]:
dicc_TipoDocumento = {
    "CEDULA DE CIUDADANIA": "CC", "CEDULA DE EXTRANJERIA": "CE, PASAPORTE, PEP O VISA", 
    "IDENTIDAD EXTRANJERA": "CE, PASAPORTE, PEP O VISA", "PASAPORTE": "CE, PASAPORTE, PEP O VISA", 
    "PERMISO DE PERMANENCIA": "CE, PASAPORTE, PEP O VISA", "TARJETA DE MOVILIDAD FRONTERIZA": "CE, PASAPORTE, PEP O VISA", 
    "TARJETA EXTRANJERIA": "CE, PASAPORTE, PEP O VISA", "VISA": "CE, PASAPORTE, PEP O VISA", 
    "CODIGO DANE DE IDENTIFICACIÓN ESTABLECIM": "DANE, NIT O RUT", 
    "NUMERO DE IDENTIFICACION TRIBUTARIA": "DANE, NIT O RUT", "REGISTRO UNICO TRIBUTARIO": "DANE, NIT O RUT", 
    "None": "ND", "NUMERO DE IDENTIFICACION NULO": "ND", "SIN DOCUMENTO": "ND", 
    "LIBRETA MILITAR": "ND", "NUMERO UNICO DE IDENTIFICACION PERSONAL": "ND", "OTRO": "ND", 
    "PARTIDA DE BAUTISMO": "RC o PB", "REGISTRO CIVIL": "RC o PB", "TARJETA DE IDENTIDAD": "TI"
}

In [0]:
# Aplicamos la función para la variable "TipoDocumentoPeticionario"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "TipoDocumentoPeticionario", diccionario=dicc_TipoDocumento
)
# Reemplazamos los valores nulos por categoría "ND"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill(value="ND",subset=["TipoDocumentoPeticionario"])
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("TipoDocumentoPeticionario").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable TipoDocumentoPeticionario están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+-------------------------+------+
TipoDocumentoPeticionario|count |
+-------------------------+------+
ND |712190|
CC |396052|
DANE, NIT O RUT |29873 |
TI |17532 |
CE, PASAPORTE, PEP O VISA|5486 |
RC o PB |2110 |
+-------------------------+------+

In [0]:
# Aplicamos la función para la variable "TipoDocumentoAfectado"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "TipoDocumentoAfectado", diccionario=dicc_TipoDocumento
)
# Reemplazamos los valores nulos por categoría "ND"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill(value="ND",subset=["TipoDocumentoAfectado"])
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("TipoDocumentoAfectado").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable TipoDocumentoAfectado están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+-------------------------+------+
TipoDocumentoAfectado |count |
+-------------------------+------+
TI |462199|
ND |420838|
RC o PB |261863|
CE, PASAPORTE, PEP O VISA|10100 |
CC |8235 |
DANE, NIT O RUT |8 |
+-------------------------+------+

## 5.4. Variables de "EdadPeticionario" y "EdadAfectado" - "TipoEdadAfectado"

In [0]:
# Al parecer la variable de EdadPeticionario tiene muchos 0 que deberían ser valores nulos porque la info no existe
# Revisemos primero los conteos de frecuencia de esta variable para valores muy bajos, muy altos o nulos
DF_DENUNCIAS_limpiovacios.filter(
    (DF_DENUNCIAS_limpiovacios["EdadPeticionario"] < 10) | \
    (DF_DENUNCIAS_limpiovacios["EdadPeticionario"] > 90) | \
    (DF_DENUNCIAS_limpiovacios["EdadPeticionario"].isNull())
).groupby("EdadPeticionario").count().sort("count", ascending=False).show()

+----------------+-------+
EdadPeticionario| count|
+----------------+-------+
 0|1038705|
 null| 51908|
 1| 324|
 9| 276|
 111| 261|
 112| 214|
 8| 213|
 7| 135|
 106| 80|
 2| 78|
 5| 68|
 6| 66|
 120| 59|
 4| 53|
 3| 51|
 119| 28|
 99| 17|
 118| 11|
 1980| 4|
 321| 3|
+----------------+-------+
only showing top 20 rows

In [0]:
# Para la variable anterior se pondrán como nulos todos los valores que sean menores de 2 (niños que no pueden hablar o escribir no registrarían una petición) o mayores de 112 (valores poco frecuentes de la edad pero atípicos)
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadPeticionario", \
            when((DF_DENUNCIAS_limpiovacios["EdadPeticionario"] < 2) | \
                (DF_DENUNCIAS_limpiovacios["EdadPeticionario"] > 112), None) \
            .otherwise(DF_DENUNCIAS_limpiovacios["EdadPeticionario"]))
#Graficamos de nuevo edades más frecuentes y frecuencias relativas
total = DF_DENUNCIAS_limpiovacios.count()
DF_DENUNCIAS_limpiovacios.groupby("EdadPeticionario").count().withColumn('percentage', F.round(((col("count")/float(total))*100),3)).sort("count", ascending=False).show()

+----------------+-------+----------+
EdadPeticionario| count|percentage|
+----------------+-------+----------+
 null|1091051| 93.794|
 30| 3357| 0.289|
 32| 2958| 0.254|
 31| 2777| 0.239|
 35| 2677| 0.23|
 29| 2661| 0.229|
 33| 2642| 0.227|
 28| 2605| 0.224|
 27| 2596| 0.223|
 34| 2498| 0.215|
 36| 2299| 0.198|
 26| 2292| 0.197|
 25| 2292| 0.197|
 40| 2051| 0.176|
 38| 2045| 0.176|
 37| 2038| 0.175|
 24| 1970| 0.169|
 23| 1827| 0.157|
 39| 1805| 0.155|
 42| 1599| 0.137|
+----------------+-------+----------+
only showing top 20 rows

In [0]:
# La mayoría son registros nulos, por lo cual para tener esta variable completa se creará una variable categórica con las categorías "ND" y los deciles de los datos que si existen
deciles = DF_DENUNCIAS_limpiovacios.approxQuantile("EdadPeticionario", [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 0)
deciles

Out[40]: [17.0, 24.0, 27.0, 30.0, 32.0, 35.0, 38.0, 42.0, 48.0]

In [0]:
# Creamos las categorías de acuerdo con los deciles
# Dependiendo si estamos creando el modelo o haciendo una predicción con nuevos datos, vamos a comentar una de los siguientes grupos de lineas. Esto debido a que cuando se está haciendo predicción no se pueden utilizar los deciles de la celda anterior para crear las categorías
# Para creación de modelo
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadPeticionario", \
            when(col("EdadPeticionario").isNull(), "ND"). \
            when((col("EdadPeticionario") <= deciles[0]), "17omenos"). \
            when((col("EdadPeticionario").between(deciles[0], deciles[1])), "18_24"). \
            when((col("EdadPeticionario").between(deciles[1], deciles[2])), "25_27"). \
            when((col("EdadPeticionario").between(deciles[2], deciles[3])), "28_30"). \
            when((col("EdadPeticionario").between(deciles[3], deciles[4])), "31_32"). \
            when((col("EdadPeticionario").between(deciles[4], deciles[5])), "33_35"). \
            when((col("EdadPeticionario").between(deciles[5], deciles[6])), "36_38"). \
            when((col("EdadPeticionario").between(deciles[6], deciles[7])), "39_42"). \
            when((col("EdadPeticionario").between(deciles[7], deciles[8])), "43_48"). \
            when((col("EdadPeticionario") > deciles[8]), "49omayor"). \
            otherwise("Error"))
# Para predicción con nuevos datos
# deciles = [17.0, 24.0, 27.0, 30.0, 32.0, 35.0, 38.0, 42.0, 48.0]
# DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadPeticionario", \
#             when(col("EdadPeticionario").isNull(), "ND"). \
#             when((col("EdadPeticionario") <= deciles[0]), "17omenos"). \
#             when((col("EdadPeticionario").between(deciles[0], deciles[1])), "18_24"). \
#             when((col("EdadPeticionario").between(deciles[1], deciles[2])), "25_27"). \
#             when((col("EdadPeticionario").between(deciles[2], deciles[3])), "28_30"). \
#             when((col("EdadPeticionario").between(deciles[3], deciles[4])), "31_32"). \
#             when((col("EdadPeticionario").between(deciles[4], deciles[5])), "33_35"). \
#             when((col("EdadPeticionario").between(deciles[5], deciles[6])), "36_38"). \
#             when((col("EdadPeticionario").between(deciles[6], deciles[7])), "39_42"). \
#             when((col("EdadPeticionario").between(deciles[7], deciles[8])), "43_48"). \
#             when((col("EdadPeticionario") > deciles[8]), "49omayor"). \
#             otherwise("Error"))
# Revisamos de nuevo frecuencias
DF_DENUNCIAS_limpiovacios.groupby("EdadPeticionario").count().withColumn('percentage', F.round(((col("count")/float(total))*100),3)).sort("count", ascending=False).show()

+----------------+-------+----------+
EdadPeticionario| count|percentage|
+----------------+-------+----------+
 ND|1091051| 93.794|
 18_24| 8789| 0.756|
 28_30| 8623| 0.741|
 33_35| 7817| 0.672|
 17omenos| 7494| 0.644|
 25_27| 7180| 0.617|
 49omayor| 7041| 0.605|
 39_42| 6736| 0.579|
 43_48| 6395| 0.55|
 36_38| 6382| 0.549|
 31_32| 5735| 0.493|
+----------------+-------+----------+

In [0]:
# Es necesario homologar las variables "EdadAfectado" - "TipoEdadAfectado" ya que de acuerdo a la segunda pueden 
# estar en años, meses, días o valores no válidos (12, Femenino, Masculino, No Aplica, None)
# Para ello se va a calcular la columna "EdadAfectado_Anios"
TipoEdadErroneo = ["12", "Femenino", "Masculino", "No Aplica"]
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Anios", \
            when(DF_DENUNCIAS_limpiovacios["TipoEdadAfectado"].isin(TipoEdadErroneo), None). \
            when(DF_DENUNCIAS_limpiovacios["TipoEdadAfectado"] == "Años", col("EdadAfectado")). \
            when(DF_DENUNCIAS_limpiovacios["TipoEdadAfectado"] == "Meses", F.round((col("EdadAfectado")/12),3)). \
            when(DF_DENUNCIAS_limpiovacios["TipoEdadAfectado"] == "Días", F.round((col("EdadAfectado")/365),3)). \
            otherwise(None)) # Este otherwise sirve para identificar errores cuando "TipoEdadAfectado" no entra en ninguna de las cat enteriores

In [0]:
# Se observan frecuencias de valores de los 50 más frecuentes para observar posibles atípicos
DF_DENUNCIAS_limpiovacios.groupby("EdadAfectado_Anios").count().sort("count", ascending=False).show(50)

+------------------+------+
EdadAfectado_Anios| count|
+------------------+------+
 0.0|248004|
 13.0| 72798|
 14.0| 67901|
 12.0| 63757|
 15.0| 56019|
 10.0| 55616|
 8.0| 51916|
 7.0| 49536|
 11.0| 48648|
 9.0| 47812|
 1.0| 47722|
 2.0| 46085|
 16.0| 45486|
 3.0| 45126|
 5.0| 44352|
 6.0| 43269|
 4.0| 42234|
 17.0| 29187|
 0.083| 4852|
 0.167| 4646|
 0.5| 4100|
 0.25| 4061|
 0.667| 4057|
 0.333| 3698|
 0.583| 3259|
 0.417| 3240|
 0.75| 3196|
 0.833| 2974|
 0.003| 2928|
 0.917| 2707|
 0.005| 1119|
 0.008| 775|
 0.014| 673|
 18.0| 644|
 0.011| 592|
 0.027| 528|
 0.022| 485|
 0.041| 482|
 0.016| 441|
 20.0| 398|
 19.0| 376|
 0.055| 371|
 0.019| 336|
 0.03| 300|
 0.033| 293|
 0.025| 285|
 0.036| 285|
 22.0| 265|
 21.0| 255|
 0.038| 242|
+------------------+------+
only showing top 50 rows

In [0]:
# Con el resultado anterior, es claro que todos los valores iguales a cero deben ser reemplazados por nulos, ya que ni siquiera registran 1 día de edad
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Anios", \
            when((col("EdadAfectado_Anios") == 0), None) \
            .otherwise(col("EdadAfectado_Anios")))

In [0]:
# Calcular años entre la fecha de registro de la petición y fecha de nacimiento del afectado
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Calculada", F.round(F.months_between(col("fechaRegistroPeticion"),col("FechaNacAfectado"))/F.lit(12),4))

In [0]:
# Calcular años entre la fecha de registro de la petición y fecha de nacimiento del afectado
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Calculada", F.round(F.months_between(col("fechaRegistroPeticion"),col("FechaNacAfectado"))/F.lit(12),3))


# Si la edad calculada es mayor o igual a 5, redondear al entero más cercano. Si es negativa, dejar un valor nulo. Si es igual a 0, dejar un valor de 0.003 (1 día de nacimiento)
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Calculada",
                 when(col("EdadAfectado_Calculada") >= 5, F.round(col("EdadAfectado_Calculada"),0)). \
                 when(col("EdadAfectado_Calculada") < 0, None). \
                 when(col("EdadAfectado_Calculada") == 0, 0.003). \
                 otherwise(col("EdadAfectado_Calculada")))

# Reemplazar los valores nulos de "EdadAfectado_Anios" con los valores calculados en "EdadAfectado_Calculada"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn('EdadAfectado_Anios', F.coalesce('EdadAfectado_Anios', 'EdadAfectado_Calculada'))
# Hacemos de nuevo una frecuencia para observar el cambio. Encontramos que los valores nulos bajaron de 262.218 a 37.049, una disminución significativa.
DF_DENUNCIAS_limpiovacios.groupby("EdadAfectado_Anios").count().sort("count", ascending=False).show(50)

+------------------+-----+
EdadAfectado_Anios|count|
+------------------+-----+
 13.0|89298|
 14.0|87216|
 12.0|75892|
 15.0|74279|
 10.0|65130|
 16.0|61474|
 8.0|61406|
 7.0|59930|
 11.0|58710|
 9.0|57152|
 6.0|52990|
 5.0|49082|
 1.0|47749|
 2.0|46118|
 3.0|45155|
 4.0|42260|
 17.0|41218|
 null|35744|
 0.083| 4901|
 18.0| 4881|
 0.167| 4698|
 0.5| 4128|
 0.25| 4096|
 0.667| 4077|
 0.003| 4006|
 0.333| 3731|
 0.583| 3286|
 0.417| 3270|
 0.75| 3221|
 0.833| 2999|
 0.917| 2733|
 0.005| 1808|
 0.008| 1363|
 0.011| 1036|
 0.016| 793|
 0.022| 711|
 0.027| 705|
 0.014| 673|
 19.0| 658|
 0.019| 653|
 20.0| 557|
 0.041| 482|
 0.03| 440|
 0.013| 423|
 21.0| 388|
 22.0| 374|
 0.055| 371|
 0.038| 354|
 23.0| 314|
 0.033| 293|
+------------------+-----+
only showing top 50 rows

In [0]:
# También se usará esta técnica para reemplazar los valores de edad afectado > 18 años, ya que potencialmente también podrían ser errores de digitación.
# Veamos una tabla de frecuencia entre las columnas "EdadAfectado_Anios" y "EdadAfectado_Calculada" para los mayores de 18
mayores18 = DF_DENUNCIAS_limpiovacios.filter(col("EdadAfectado_Anios") > 18)
print("# registros con edad mayor a 18 años: ", mayores18.count())
mayores18.groupby("EdadAfectado_Anios", "EdadAfectado_Calculada").count().sort("EdadAfectado_Anios", ascending=False).show(400)

# registros con edad mayor a 18 años: 5610
+------------------+----------------------+-----+
EdadAfectado_Anios|EdadAfectado_Calculada|count|
+------------------+----------------------+-----+
 12320.0| 11.0| 1|
 2004.0| 14.0| 1|
 2004.0| 13.0| 1|
 200.0| 14.0| 1|
 161.0| 17.0| 1|
 155.0| 16.0| 1|
 147.0| 9.0| 1|
 147.0| null| 1|
 145.0| 16.0| 1|
 145.0| 14.0| 1|
 145.0| 15.0| 1|
 140.0| 3.242| 1|
 136.0| 17.0| 1|
 136.0| 13.0| 1|
 133.0| 14.0| 1|
 131.0| 14.0| 1|
 127.0| 17.0| 1|
 123.0| 13.0| 1|
 121.0| 121.0| 3|
 120.0| 120.0| 1|
 118.0| 7.0| 1|
 115.0| 14.0| 1|
 115.0| null| 2|
 115.0| 15.0| 1|
 114.0| 15.0| 2|
 114.0| 16.0| 1|
 113.0| 13.0| 1|
 113.0| null| 2|
 112.0| null| 1|
 112.0| 12.0| 2|
 111.0| 13.0| 1|
 111.0| 16.0| 1|
 111.0| null| 11|
 110.0| 10.0| 1|
 110.0| null| 1|
 107.0| 107.0| 1|
 107.0| null| 1|
 103.0| 93.0| 1|
 101.0| 10.0| 2|
 101.0| 1.78| 1|
 100.0| 11.0| 1|
 100.0| null| 9|
 99.0| 2.836| 1|
 99.0| 11.0| 2|
 99.0| 13.0| 1|
 99.0| 0.07| 1|
 99.0| 10.0| 1|
 99.0| 15.0| 1|
 99.0| null| 13|
 99.0| 2.599| 1|
 99.0| 4.207| 1|
 99.0| 16.0| 1|
 99.0| 17.0| 3|
 98.0| null| 1|
 97.0| null| 1|
 96.0| null| 1|
 95.0| null| 1|
 94.0| null| 2|
 93.0| null| 5|
 92.0| null| 3|
 91.0| 91.0| 2|
 90.0| 90.0| 4|
 90.0| null| 4|
 89.0| null| 2|
 89.0| 90.0| 1|
 88.0| 88.0| 2|
 88.0| null| 6|
 87.0| 87.0| 1|
 87.0| null| 11|
 86.0| null| 2|
 86.0| 86.0| 1|
 85.0| 65.0| 1|
 85.0| 85.0| 1|
 85.0| null| 6|
 84.0| null| 7|
 84.0| 83.0| 1|
 83.0| 85.0| 1|
 83.0| 84.0| 2|
 83.0| null| 3|
 82.0| null| 10|
 82.0| 82.0| 1|
 82.0| 83.0| 1|
 82.0| 89.0| 1|
 81.0| 34.0| 1|
 81.0| null| 3|
 80.0| 9.0| 1|
 80.0| 80.0| 2|
 80.0| null| 11|
 79.0| 10.0| 1|
 79.0| null| 5|
 79.0| 82.0| 1|
 78.0| null| 3|
 77.0| null| 7|
 77.0| 77.0| 2|
 76.0| null| 2|
 75.0| 75.0| 2|
 75.0| null| 6|
 75.0| 78.0| 1|
 75.0| 64.0| 1|
 74.0| 74.0| 2|
 74.0| null| 4|
 74.0| 75.0| 1|
 73.0| null| 3|
 72.0| null| 8|
 71.0| null| 5|
 70.0| 10.0| 1|
 70.0| 70.0| 1|
 70.0| 73.0| 2|
 70.0| 7.0| 1|
 70.0| 48.0| 1|
 70.0| null| 10|
 69.0| 70.0| 1|
 69.0| null| 3|
 69.0| 69.0| 1|
 68.0| 68.0| 1|
 68.0| null| 3|
 67.0| null| 3|
 67.0| 68.0| 1|
 67.0| 67.0| 1|
 66.0| 66.0| 4|
 66.0| 68.0| 1|
 66.0| null| 5|
 66.0| 69.0| 1|
 65.0| 13.0| 1|
 65.0| 65.0| 2|
 65.0| 71.0| 1|
 65.0| 67.0| 1|
 65.0| null| 6|
 64.0| 9.0| 1|
 64.0| 64.0| 3|
 64.0| 65.0| 1|
 64.0| null| 3|
 63.0| null| 3|
 63.0| 63.0| 3|
 62.0| null| 2|
 61.0| 9.0| 1|
 61.0| 10.0| 1|
 61.0| null| 2|
 61.0| 61.0| 1|
 60.0| 59.0| 1|
 60.0| 60.0| 1|
 60.0| 16.0| 1|
 60.0| 95.0| 1|
 60.0| null| 20|
 60.0| 31.0| 1|
 59.0| 11.0| 1|
 59.0| 61.0| 1|
 59.0| 2.519| 1|
 59.0| null| 7|
 59.0| 59.0| 2|
 59.0| 58.0| 1|
 58.0| 58.0| 2|
 58.0| 57.0| 2|
 58.0| null| 5|
 58.0| 59.0| 1|
 58.0| 56.0| 1|
 57.0| null| 8|
 57.0| 54.0| 1|
 57.0| 57.0| 4|
 56.0| null| 2|
 56.0| 56.0| 4|
 56.0| 57.0| 3|
 55.0| 55.0| 5|
 55.0| 14.0| 1|
 55.0| 64.0| 1|
 55.0| 34.0| 1|
 55.0| null| 10|
 54.0| 6.0| 2|
 54.0| 7.0| 1|
 54.0| 54.0| 5|
 54.0| 55.0| 1|
 54.0| 16.0| 1|
 54.0| 12.0| 1|
 54.0| null| 4|
 53.0| 13.0| 1|
 53.0| 53.0| 2|
 53.0| 17.0| 1|
 53.0| 10.0| 1|
 53.0| 7.0| 1|
 53.0| null| 3|
 52.0| 42.0| 1|
 52.0| null| 5|
 52.0| 6.0| 1|
 52.0| 58.0| 1|
 52.0| 52.0| 7|
 52.0| 56.0| 1|
 51.0| 51.0| 7|
 51.0| 48.0| 2|
 51.0| 56.0| 1|
 51.0| null| 1|
 51.0| 0.137| 1|
 51.0| 52.0| 2|
 50.0| null| 13|
 50.0| 54.0| 1|
 50.0| 52.0| 1|
 50.0| 48.0| 1|
 50.0| 55.0| 1|
 50.0| 51.0| 1|
 50.0| 46.0| 1|
 50.0| 50.0| 4|
 50.0| 13.0| 1|
 50.0| 76.0| 1|
 49.0| 49.0| 8|
 49.0| 0.156| 1|
 49.0| null| 8|
 49.0| 50.0| 3|
 49.0| 17.0| 1|
 48.0| 10.0| 1|
 48.0| 49.0| 2|
 48.0| 16.0| 1|
 48.0| 48.0| 11|
 48.0| 12.0| 1|
 48.0| null| 7|
 48.0| 11.0| 1|
 48.0| 13.0| 1|
 47.0| 1.836| 1|
 47.0| 48.0| 2|
 47.0| 50.0| 1|
 47.0| 49.0| 1|
 47.0| 7.0| 1|
 47.0| 46.0| 1|
 47.0| null| 8|
 47.0| 4.078| 1|
 47.0| 47.0| 5|
 46.0| 45.0| 2|
 46.0| 12.0| 1|
 46.0| null| 6|
 46.0| 48.0| 1|
 46.0| 47.0| 2|
 46.0| 46.0| 15|
 46.0| 3.925| 1|
 45.0| 12.0| 1|
 45.0| 15.0| 1|
 45.0| 46.0| 1|
 45.0

In [0]:
# Se encuentra que para edades muy altas hay multiples casos en que los que puede haber errores de digitación y es una mejor información la edad calculada. Haremos el reemplazo solo bajo ciertas condiciones
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn('EdadAfectado_Anios', \
                            when(((col('EdadAfectado_Anios') > 18) & \
                                (col('EdadAfectado_Anios') > col('EdadAfectado_Calculada')) & \
                                (col('EdadAfectado_Calculada').isNotNull()) & \
                                (col('EdadAfectado_Calculada') > 1) & \
                                (col('EdadAfectado_Calculada') < 21)), 
                                    col('EdadAfectado_Calculada')). \
                            otherwise(col("EdadAfectado_Anios")))

# Luego volvemos a contar los registros mayores a 18 años. Vemos que hubo cerca de 400 registros arreglados
mayores18 = DF_DENUNCIAS_limpiovacios.filter(col("EdadAfectado_Anios") > 18)
print("# registros con edad mayor a 18 años: ", mayores18.count())

# registros con edad mayor a 18 años: 5269

In [0]:
# Volvemos a hacer una tabla observando la frecuencia de los 50 valores más altos en EdadAfectado
print(DF_DENUNCIAS_limpiovacios.groupby("EdadAfectado_Anios").count().sort("EdadAfectado_Anios", ascending=False).show(20))

# Se encuentra que hay multiples registros con edades muy altas y no han sido corregidos con la técnica anterior. A todas las edades mayores a 98 se pondrán valores nulos
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Anios", \
            when((col("EdadAfectado_Anios") > 98), None) \
            .otherwise(col("EdadAfectado_Anios")))

# Finalmente, contamos el porcentaje de registros nulos para dimensionar una estrategia de imputación en esta columna tan importante
total = DF_DENUNCIAS_limpiovacios.count()
nulos = DF_DENUNCIAS_limpiovacios.filter(col("EdadAfectado_Anios").isNull()).count()
print("# de registros nulos: {} ({:.3%} del total)".format(nulos, nulos/total))

+------------------+-----+
EdadAfectado_Anios|count|
+------------------+-----+
 147.0| 1|
 121.0| 3|
 120.0| 1|
 115.0| 2|
 113.0| 2|
 112.0| 1|
 111.0| 11|
 110.0| 1|
 107.0| 2|
 103.0| 1|
 100.0| 9|
 99.0| 14|
 98.0| 1|
 97.0| 1|
 96.0| 1|
 95.0| 1|
 94.0| 2|
 93.0| 5|
 92.0| 3|
 91.0| 2|
+------------------+-----+
only showing top 20 rows

None
# de registros nulos: 35792 (3.077% del total)

In [0]:
# Se eliminan las variables de edad que ahora no son necesarias
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.drop("EdadAfectado_Calculada", "TipoEdadAfectado", "EdadAfectado")

In [0]:
# Se crea una variable de EdadAfectado_Agrupado, en la que se redondea la edad al entero menor (floor) y se crea el grupo "18_o_mas"
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Agrupado",
                                                                 when(col("EdadAfectado_Anios")<18, F.floor(col("EdadAfectado_Anios")).cast("string")).
                                                                when(col("EdadAfectado_Anios")>=18, "18_o_mas").
                                                                otherwise(col("EdadAfectado_Anios")))

In [0]:
# Se creará una variable de EdadAfectado_Anios en la que se imputan los valores vacíos. Se imputará el promedio calculado a partir de dos variables TipoDocumentoAfectado y MotivoPeticion
valores_imputacion = DF_DENUNCIAS_limpiovacios.groupBy("TipoDocumentoAfectado", "MotivoPeticion"). \
                      agg(F.floor(F.mean('EdadAfectado_Anios'))). \
                      select(col("TipoDocumentoAfectado").alias("TipoDocumentoAfectado2"), 
                             col("MotivoPeticion").alias("MotivoPeticion2"), 
                             col("FLOOR(avg(EdadAfectado_Anios))").alias("Edad_Imputar"))

# Se realiza el join con el df principal
cond = [DF_DENUNCIAS_limpiovacios["TipoDocumentoAfectado"] == valores_imputacion["TipoDocumentoAfectado2"],
        DF_DENUNCIAS_limpiovacios["MotivoPeticion"] == valores_imputacion["MotivoPeticion2"]]

DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.join(valores_imputacion, cond, how='left')

# Se crea columna con valores imputados
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn("EdadAfectado_Anios_Imputada", col("EdadAfectado_Anios"))
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.withColumn('EdadAfectado_Anios_Imputada', F.coalesce('EdadAfectado_Anios', 'Edad_Imputar'))

# Eliminar columnas innecesarias
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.drop("TipoDocumentoAfectado2", "MotivoPeticion2", "Edad_Imputar")

In [0]:
display(valores_imputacion)

TipoDocumentoAfectado2,MotivoPeticion2,Edad_Imputar
RC o PB,Maltrato,3
CC,Conductas_Adolescentes,15
TI,Trata,13
"CE, PASAPORTE, PEP O VISA",Conductas_Adolescentes,11
TI,Conductas_Adolescentes,13
"CE, PASAPORTE, PEP O VISA",NNA_solo,13
ND,Otro,6
ND,Abandono_Amenaza,7
ND,Maltrato,7
"CE, PASAPORTE, PEP O VISA",PI_Vulnerable,17


## 5.5. Variables de sexo "SexoPeticionario" y "SexoAfectado"

In [0]:
dicc_Sexo = {
    "Femenino": "F","Masculino": "M","Años": "ND",
    "No Aplica": "ND","No Reporta": "ND",
    "NO SE AUTORRECONOCE EN NINGUNO DE LOS ANTERIORES": "ND","None": "ND"
}

In [0]:
# Aplicamos la función para la variable "SexoPeticionario"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "SexoPeticionario", diccionario=dicc_Sexo
)
# Para los valores nulos se requiere hacer un reemplazo adicional
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill("ND", subset="SexoPeticionario")
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("SexoPeticionario").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable SexoPeticionario están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+----------------+------+
SexoPeticionario|count |
+----------------+------+
ND |516553|
F |482114|
M |164576|
+----------------+------+

In [0]:
# Aplicamos la función para la variable "SexoAfectado"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "SexoAfectado", diccionario=dicc_Sexo
)
# Para los valores nulos se requiere hacer un reemplazo adicional
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill("ND", subset="SexoAfectado")
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("SexoAfectado").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable SexoAfectado están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+------------+------+
SexoAfectado|count |
+------------+------+
F |610718|
M |515306|
ND |37219 |
+------------+------+

## 5.6. Variables de Grupo Étnico "GrupoEtnicoPeticionario" y "GrupoEtnicoAfectado"

In [0]:
dicc_GrupoEtnico = {
    "AFROCOLOMBIANO (A)": "AFROCOLOMBIANO (A)","COMUNIDAD NEGRA": "AFROCOLOMBIANO (A)",
    "NEGRO": "AFROCOLOMBIANO (A)","INDÍGENA": "INDÍGENA","Apuntador": "ND",
    "Masculino": "ND","N": "ND","N-D": "ND",
    "NELIDA IRIS SERRANO MENDOZA": "ND","NINGUNO": "ND",
    "NO SE AUTORRECONOCE EN NINGUNO DE LOS ANTERIORES": "ND",
    "None": "ND","Sin pertenencia étnica": "ND",
    "PALENQUERO": "PALENQUERO (A)","PALENQUERO (A)": "PALENQUERO (A)","RAIZAL": "RAIZAL",
    "san andres": "RAIZAL", "san andres ": "RAIZAL","ROM/GITANO": "ROM/GITANO"
}

In [0]:
# Aplicamos la función para la variable "GrupoEtnicoPeticionario"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "GrupoEtnicoPeticionario", diccionario=dicc_GrupoEtnico
)
# Para los valores nulos se requiere hacer un reemplazo adicional
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill("ND", subset="GrupoEtnicoPeticionario")
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("GrupoEtnicoPeticionario").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable GrupoEtnicoPeticionario están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+-----------------------+-------+
GrupoEtnicoPeticionario|count |
+-----------------------+-------+
ND |1133375|
AFROCOLOMBIANO (A) |19111 |
INDÍGENA |9121 |
RAIZAL |1401 |
PALENQUERO (A) |187 |
ROM/GITANO |48 |
+-----------------------+-------+

In [0]:
# Aplicamos la función para la variable "GrupoEtnicoAfectado"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "GrupoEtnicoAfectado", diccionario=dicc_GrupoEtnico
)
# Para los valores nulos se requiere hacer un reemplazo adicional
DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill("ND", subset="GrupoEtnicoAfectado")
# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("GrupoEtnicoAfectado").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable GrupoEtnicoAfectado están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+-------------------+-------+
GrupoEtnicoAfectado|count |
+-------------------+-------+
ND |1140134|
AFROCOLOMBIANO (A) |11948 |
INDÍGENA |8671 |
RAIZAL |2396 |
PALENQUERO (A) |78 |
ROM/GITANO |16 |
+-------------------+-------+

## 5.7. Se hace un conjunto de reemplazos de variables que tienen valores inválidos o representan nulos
#### "PuebloIndigenaPeticionario", "PresentaDiscapacidadPeticionario", "CondicionDesplazamientoPeticionario", "HaSidoVictimaPeticionario", "DetalleZonAfectado", "PuebloIndigenaAfectado", "PresentaDiscapacidadAfectado", "CualDiscapacidadAfectado", "CondicionDesplazamientoAfectado"

In [0]:
#Se crea listado de variables que se puede homologar en conjunto valores inválidos o nulos
lista_var_homologar = ["PuebloIndigenaPeticionario", "PresentaDiscapacidadPeticionario", 
                       "CondicionDesplazamientoPeticionario", "HaSidoVictimaPeticionario", 
                       "DetalleZonAfectado", "PuebloIndigenaAfectado", "PresentaDiscapacidadAfectado", 
                       "CualDiscapacidadAfectado", "CondicionDesplazamientoAfectado"]

In [0]:
dicc_Nulos = {
    "N": None, "A": None, "No": None, "01/01/1900 0:00": None, "01/01/2004 0:00": None, "02/04/2007 0:00": None, 
    "03/03/1997 0:00": None, "07/11/2011 0:00": None, "19/09/2003 0:00": None, "2017-10-04 00:00:00.000": None, 
    "31/03/2001 0:00": None, "NO SE AUTORRECONOCE EN NINGUNO DE LOS ANTERIORES": None, 
    "Hijos de Padres que se Encuentran Privados de la Libertad por Orden Judicial": None, 
    "Omisión o negligencia": None, "Por Condiciones Especiales de Cuidadores": None
}

In [0]:
# Reemplazamos los valores de estas variables
for i in lista_var_homologar:
    DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.replace(dicc_Nulos, subset=i)
    DF_DENUNCIAS_limpiovacios = DF_DENUNCIAS_limpiovacios.na.fill("ND", subset=i)
# Verificamos que todas las variables en la lista ahora tengan valores corregidos
for i in lista_var_homologar:
    DF_DENUNCIAS_limpiovacios.groupby(i).count().sort(desc("count")).show(truncate=False)

+--------------------------+-------+
PuebloIndigenaPeticionario|count |
+--------------------------+-------+
ND |1162397|
S |846 |
+--------------------------+-------+

+--------------------------------+-------+
PresentaDiscapacidadPeticionario|count |
+--------------------------------+-------+
ND |1157772|
Si |5471 |
+--------------------------------+-------+

+-----------------------------------+-------+
CondicionDesplazamientoPeticionario|count |
+-----------------------------------+-------+
ND |1116863|
S |46380 |
+-----------------------------------+-------+

+-------------------------+-------+
HaSidoVictimaPeticionario|count |
+-------------------------+-------+
ND |1136424|
S |26819 |
+-------------------------+-------+

+------------------+-------+
DetalleZonAfectado|count |
+------------------+-------+
Urbana |1075175|
Rural |88064 |
ND |3 |
Dispersa |1 |
+------------------+-------+

+----------------------+-------+
PuebloIndigenaAfectado|count |
+----------------------+-------+
ND |1155073|
S |8170 |
+----------------------+-------+

+----------------------------+-------+
PresentaDiscapacidadAfectado|count |
+----------------------------+-------+
ND |1160856|
Si |2387 |
+----------------------------+-------+

+-----------------------------+-------+
CualDiscapacidadAfectado |count |
+-----------------------------+-------+
ND |1161155|
Mental Cognitiva |910 |
Mental Psicosocial |408 |
Movilidad |287 |
Presenta más de una Categoria|234 |
Sensorial Auditiva |67 |
Voz y Habla |62 |
Sensorial Visual |59 |
Sistémica |52 |
Sensorial Sordo Ceguera |4 |
Sensorial Gusto Olfato |3 |
Piel Pelo y Uñas |2 |
+-----------------------------+-------+

+-------------------------------+-------+
CondicionDesplazamientoAfectado|count |
+-------------------------------+-------+
ND |1159413|
S |3830 |
+-------------------------------+-------+

## 5.8. Variable "UNFICADO_MOTIVO_INGRESO"

In [0]:
dicc_UNFICADO_MOTIVO_INGRESO = {
    "Victima de Violencia Sexual-Explotación Sexual Comercial": "ESCNNA", "Maltrato": "Maltrato", 
    "Omisión o negligencia": "Maltrato", "Violencia física": "Maltrato", "Violencia Psicológica": "Maltrato", 
    "Abandono": "Otro", "Acoso Escolar, Matoneo o Bullying": "Otro", "Alta permanencia en calle": "Otro", 
    "Amenazados Contra de su Vida por Acción de los Grupos Armados Organizados al Margen de la Ley": "Otro", 
    "Amenazados de Reclutamiento Inminente por parte de los Grupos Armados Organizados al Margen de la Ley": "Otro", 
    "Apoyo a NNA afectados por emergencias o desastres naturales": "Otro", 
    "Competencia declaratoria adoptabilidad.": "Otro", "Conductas Sexuales entre Menores de 14 años": "Otro", 
    "Consentimiento para adopción del hijo por cónyuge o compañero": "Otro", 
    "Consentimiento para adopción por consanguíneo": "Otro", "Consentimiento para adopción.": "Otro", 
    "Convivencia Educativa": "Otro", "Desnutrición": "Otro", "Desplazamiento Forzado": "Otro", "Expósito": "Otro", 
    "Extraviado": "Otro", "Falta absoluta o temporal de responsables": "Otro", 
    "Hijo (a) de gestante / lactante mayor de 18 años en protección": "Otro", "Hijos de adolescentes en PARD": "Otro", 
    "Hijos de Padres que se Encuentran Privados de la Libertad por Orden Judicial": "Otro", 
    "Hijos e hijas de víctimas directas de trata": "Otro", 
    "Huérfanos a Causa de la Violencia Armada, Hijos de Padres Desaparecidos o Secuestrados por Acción de los Grupos Armados Organizados al Margen de la Ley": "Otro", 
    "Inmigrante": "Otro", "Menor de 14 Años en Comisión de un Delito": "Otro", "Menor de 14 años Gestante": "Otro", 
    "Menor de 14 años Lactante": "Otro", "Menor de 18 Años en Situación de Discapacidad en Comisión de un Delito": "Otro", 
    "Mujer en Gestación o Lactancia en Riesgo": "Otro", "Mutilación genital femenina": "Otro", 
    "Niños, niñas y adolescentes migrantes no acompañados": "Otro", 
    "Niños, niñas, adolescentes nacidos como consecuencia del abuso sexual en el marco de conflicto armado.": "Otro", 
    "No Reclamado en Tiempo Razonable": "Otro", "Por Condiciones Especiales de Cuidadores": "Otro", 
    "Problemas del Consumo de Sustancias Psicoactivas": "Otro", "Retención Arbitraria": "Otro", 
    "Reunificación Familiar": "Otro", "Situación de Amenaza a la Integridad": "Otro", "Situación de Emergencia": "Otro", 
    "Situación de vida en calle": "Otro", 
    "Uso y/o Utilización de niño, niña o adolescente para la comisión de un delito.": "Otro", 
    "Victima de Minas Antipersonal, Municiones Sin Explotar o Artefacto Explosivo Improvisado": "Otro", 
    "Victima de Ola Invernal": "Otro", "Victima de uso, porte, manipulación o lesión por pólvora": "Otro", 
    "Victima Otros Delitos": "Otro", 
    "Víctimas de Acto Terrorista - Atentados - Combates - Enfrentamientos - Hostigamientos": "Otro", 
    "Violencia Intrafamiliar": "Otro", "Vulneración a la Intimidad": "Otro",  "Vulneración  a la Intimidad": "Otro",
    "None": "Sinvulner", 
    "Explotación Laboral": "TI", "Seguimiento al Trabajo Adolescente": "TI", "Trabajo Infantil": "TI", 
    "Trata de Personas - Explotación Sexual": "Trata de personas", 
    "Trata de personas – Fines de servidumbre": "Trata de personas", 
    "Trata de personas – Fines de Trabajo": "Trata de personas", 
    "Trata de personas – Matrimonio Servil": "Trata de personas", 
    "Trata de personas – Mendicidad Ajena": "Trata de personas", "Acceso Carnal": "V_sexual", 
    "Acoso Sexual": "V_sexual", "Actos Sexuales": "V_sexual", "Otras formas de violencia sexual": "V_sexual", 
    "Unión o matrimonio temprano": "V_sexual", "Victima de Violencia Sexual-Abuso Sexual": "V_sexual", 
    "Víctimas de violencia sexual en el marco de conflicto armado": "V_sexual", "Violación / Asalto Sexual": "V_sexual"
}

In [0]:
# Aplicamos la función para la variable "GrupoEtnicoAfectado"
DF_DENUNCIAS_limpiovacios = check_replace_llaves(
    df = DF_DENUNCIAS_limpiovacios, var = "UNFICADO_MOTIVO_INGRESO", diccionario=dicc_UNFICADO_MOTIVO_INGRESO
)

# Verificamos que el reemplazo se haya hecho correctamente
DF_DENUNCIAS_limpiovacios.groupby("UNFICADO_MOTIVO_INGRESO").count().sort(desc("count")).show(truncate=False)

True = Todas las categorías de la variable UNFICADO_MOTIVO_INGRESO están incluidas en el diccionario. 
 Se hizo el reemplazo de los valores de acuerdo al diccionario.
+-----------------------+------+
UNFICADO_MOTIVO_INGRESO|count |
+-----------------------+------+
null |896285|
Otro |116530|
V_sexual |75413 |
Maltrato |65044 |
TI |7695 |
ESCNNA |1840 |
Trata de personas |436 |
+-----------------------+------+

In [0]:
pd.set_option("max_rows", 200)
hdf = DF_DENUNCIAS_limpiovacios.toHandy()
a = hdf.isnull(ratio=False)

# 7. Se guarda el archivo final en la instacia SQL y en formato parquet

In [0]:
# Luego de los ajustes escribimos los datos a la instancia SQL
jdbcHostname = dbutils.secrets.get('blobsecret','SQLHostname')
jdbcDatabase = dbutils.secrets.get('blobsecret','SQLDatabase')
jdbcPort = dbutils.secrets.get('blobsecret','SQLport')
jdbcuser = dbutils.secrets.get('blobsecret','SQLusername')
jdbcpass = dbutils.secrets.get('blobsecret','SQLpassword')
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcuser,
  "password" : jdbcpass,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

DF_DENUNCIAS_limpiovacios.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("url", jdbcUrl).option("dbtable", 'DENUNCIAS2015_2021_31032021_BASELIMPIA').option("user", jdbcuser).option("password", jdbcpass).save()

In [0]:
# Configuración necesaria para poder guardar archivos en el storage account
spark.conf.set ( "fs.azure.account.key.dlsicbfdenuncias.dfs.core.windows.net",
  "LzRpBlOfsiEWD+I6HnUGqQWvLbK4v2IWLc+205FSuhimA8mXWj40UgunJkt2kmMBUAndRTrGuYw2BBM3WVMsIQ=="
)

file_location = "abfss://basecruzada@dlsicbfdenuncias.dfs.core.windows.net/baselimpia.parquet"
DF_DENUNCIAS_limpiovacios.coalesce(1).write.option("header", "true").mode("overwrite").parquet(file_location)

# 8. Se construye diccionario de datos

In [0]:
def describe_spark_df(df, debug=False):
    start_time = datetime.now() # Para contabilizar cuánto se demora el proceso
    diccionario = [] # Inicializar el diccionario en que se guardarán los datos
    len_df = df.count() # Número de filas que tiene el df
    n_cols_df = len(df.columns) # Número de columnas que tiene el df
    series_null = df.toHandy().isnull(ratio=False) # Esta funcion nos permite contar rápidamente los nulos
    
    for contador, i in enumerate(df.columns):
        if debug: # En caso de errores en el código, debug=True nos dice en cuál variable se ocasiona el problema
            print(contador, i) 
            
        valores = cuenta_valores = moda_val = media_val = ds_val = minimo_val \
        = p25_val = mediana_val = p75_val = maximo_val = "" # Inicializar outputs
        
        df_column = df.select(i)  # Seleccionamos solo la columna que esta corriendo en el loop
        tipo_dato = df_column.dtypes[0][1] # Qué tipo de dato es la columna
        largo_col = df_column.withColumn('largo_col', \
                    F.length(i)).agg({"largo_col": "max"}).collect()[0]["max(largo_col)"] # Máximo largo carácteres columna
        
        null_val = series_null.loc[i] # Recuperamos el número de nulos que tiene la columna
        porc_null = null_val / len_df # Porcentaje de nulos
        
        # Cuenta de valores únicos
        cuenta_unicos = df_column.agg(F.countDistinct(df_column[i]).alias('Unicos')).collect()[0]['Unicos']
        
        if cuenta_unicos < 100:
            df_cuenta_valores = df_column.groupBy(i).count().sort(desc("count")).collect()
            moda_val = str(df_cuenta_valores[0][i]) + " - " + str(df_cuenta_valores[0]["count"])
            
            lista_valores = [str(row[i]) for row in df_cuenta_valores]
            lista_valores2 = lista_valores.copy()
            lista_valores2.sort()
            separador = ", "
            valores = separador.join(lista_valores2)

            lista_cuenta = [str(row["count"]) for row in df_cuenta_valores]
            lista_cuentavalores = [lista_valores[j] + " - " + lista_cuenta[j] for j in range(len(lista_valores))]
            cuenta_valores = separador.join(lista_cuentavalores)
            
        if ((tipo_dato == "int") | (tipo_dato == "double")) & (largo_col!=None):
            media_val = "{:.4f}".format(df_column.agg({i: 'mean'}).collect()[0][0])
            ds_val = "{:.4f}".format(df_column.agg({i: 'stddev'}).collect()[0][0])
            minimo_val = "{:.4f}".format(df_column.agg({i: 'min'}).collect()[0][0])
            p25_val = "{:.4f}".format(df_column.agg(F.expr('percentile({}, array(0.25))'.format(i))).collect()[0][0][0])
            mediana_val = "{:.4f}".format(df_column.agg(F.expr('percentile({}, array(0.50))'.format(i))).collect()[0][0][0])
            p75_val = "{:.4f}".format(df_column.agg(F.expr('percentile({}, array(0.75))'.format(i))).collect()[0][0][0])
            maximo_val = "{:.4f}".format(df_column.agg({i: 'max'}).collect()[0][0])

        data_campo = {
            "Campo": i,
            "Tipo Dato": str(tipo_dato),
            "Longitud Caracteres": largo_col,
            "Valores nulos": "{:d}".format(null_val),
            "Porcentaje nulos": "{:.2%}".format(porc_null),
            "Número de valores únicos": "{:d}".format(cuenta_unicos),
            "Valores únicos": valores,
            "Cuenta valores únicos": cuenta_valores,
            "Moda - Freq": moda_val,
            "Media": str(media_val),
            "Desviación estándar": str(ds_val),
            "Mínimo": str(minimo_val),
            "Percentil 25%": str(p25_val),
            "Mediana": str(mediana_val),
            "Percentil 75%": str(p75_val),
            "Máximo": str(maximo_val)             
        }
        
        diccionario.append(data_campo)

        if ((contador+1) % 10 == 0) | ((contador+1) == n_cols_df):
            print("Progreso: ", ((contador+1)/n_cols_df)*100, "%. Duración: ", datetime.now() - start_time)
    df_resultado = pd.DataFrame(diccionario)
    return df_resultado

In [0]:
# Se ejecuta la función definida en la anterior celda que hace por completo la descripción de todas columnas.
# Este se guarda en un pd.DataFrame que llamamos descriptivas_limpieza
descriptivas_limpieza = describe_spark_df(DF_DENUNCIAS_limpiovacios)

Progreso: 8.620689655172415 %. Duración: 0:24:25.663029
Progreso: 17.24137931034483 %. Duración: 0:36:04.668166
Progreso: 25.862068965517242 %. Duración: 0:54:50.109161
Progreso: 34.48275862068966 %. Duración: 1:10:58.807594
Progreso: 43.103448275862064 %. Duración: 1:29:02.116796
Progreso: 51.724137931034484 %. Duración: 1:42:19.737802
Progreso: 60.3448275862069 %. Duración: 1:56:11.269324
Progreso: 68.96551724137932 %. Duración: 2:13:14.567461
Progreso: 77.58620689655173 %. Duración: 2:25:20.464911
Progreso: 86.20689655172413 %. Duración: 2:52:18.676055
Progreso: 94.82758620689656 %. Duración: 3:35:20.088931
Progreso: 100.0 %. Duración: 4:03:03.000946

In [0]:
# Convertimos el resultado de pd.DataFrame a spark.Dataframe
descriptivas_limpieza_spark = spark.createDataFrame(descriptivas_limpieza)
# Exportamos el spark.Dataframe a nuestro contenedor de Azure y lo previsualizamos acá en Databricks
file_location = "abfss://export@dlsicbfdenuncias.dfs.core.windows.net/sample/descriptivas_limpieza_final.csv"
descriptivas_limpieza_spark.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode("overwrite").save(file_location)
display(descriptivas_limpieza_spark)

Campo,Tipo Dato,Longitud Caracteres,Valores nulos,Porcentaje nulos,Número de valores únicos,Valores únicos,Cuenta valores únicos,Moda - Freq,Media,Desviación estándar,Mínimo,Percentil 25%,Mediana,Percentil 75%,Máximo
NumeroPeticion,bigint,11.0,0,0.00%,1163243,,,,,,,,,,
EsPeticionHija,string,2.0,0,0.00%,2,"NO, SI","NO - 1075721, SI - 87522",NO - 1075721,,,,,,,
fechaRegistroPeticion,date,10.0,0,0.00%,2338,,,,,,,,,,
AnoRegistroPeticion,int,4.0,0,0.00%,7,"2015, 2016, 2017, 2018, 2019, 2020, 2021","2018 - 222002, 2019 - 201073, 2017 - 191202, 2020 - 176132, 2016 - 157515, 2015 - 140481, 2021 - 74838",2018 - 222002,2017.8712,1.7703,2015.0000,2016.0000,2018.0000,2019.0000,2021.0000
MesRegistroPeticion,string,3.0,0,0.00%,12,"Apr, Aug, Dec, Feb, Jan, Jul, Jun, Mar, May, Nov, Oct, Sep","Mar - 112593, May - 111132, Feb - 105749, Sep - 103296, Apr - 101199, Aug - 100247, Oct - 98643, Jul - 94400, Nov - 89463, Jun - 86986, Jan - 86165, Dec - 73370",Mar - 112593,,,,,,,
TipoPeticion,string,47.0,0,0.00%,2,"Reporte de Amenaza o Vulneración de derechos, Solicitud de Restablecimiento de Derechos (SRD)","Solicitud de Restablecimiento de Derechos (SRD) - 841590, Reporte de Amenaza o Vulneración de derechos - 321653",Solicitud de Restablecimiento de Derechos (SRD) - 841590,,,,,,,
MotivoPeticion,string,22.0,0,0.00%,13,"Abandono_Amenaza, Conductas_Adolescentes, Desvinculados, ESCNNA, Maltrato, ND, NNA_solo, Otro, PI_Vulnerable, Permanencia_Calle, TI, Trata, V_sexual","Maltrato - 594220, V_sexual - 207041, Conductas_Adolescentes - 164712, Otro - 42773, TI - 34206, Abandono_Amenaza - 32291, NNA_solo - 25517, Permanencia_Calle - 22818, ND - 17750, ESCNNA - 9173, PI_Vulnerable - 8330, Desvinculados - 3428, Trata - 984",Maltrato - 594220,,,,,,,
RegionalCreacion,string,18.0,0,0.00%,34,"AMAZONAS, ANTIOQUIA, ARAUCA, ATLANTICO, BOGOTA, BOLIVAR, BOYACA, CALDAS, CAQUETA, CASANARE, CAUCA, CESAR, CHOCO, CORDOBA, CUNDINAMARCA, GUAINIA, GUAVIARE, HUILA, LA GUAJIRA, MAGDALENA, META, NARIÑO, NORTE DE SANTANDER, PUTUMAYO, QUINDIO, RISARALDA, SAN ANDRES, SANTANDER, SEDE NACIONAL, SUCRE, TOLIMA, VALLE DEL CAUCA, VAUPES, VICHADA","SEDE NACIONAL - 472760, BOGOTA - 113488, VALLE DEL CAUCA - 64912, ANTIOQUIA - 51212, CUNDINAMARCA - 39665, TOLIMA - 33377, NARIÑO - 29707, CALDAS - 28063, HUILA - 27241, SANTANDER - 25890, CORDOBA - 25517, ATLANTICO - 25472, NORTE DE SANTANDER - 22067, META - 21095, RISARALDA - 21016, BOYACA - 20455, BOLIVAR - 17206, CAUCA - 16369, MAGDALENA - 16286, CESAR - 13699, QUINDIO - 13048, LA GUAJIRA - 10429, CASANARE - 10258, SUCRE - 9099, CAQUETA - 7023, PUTUMAYO - 6781, CHOCO - 4418, ARAUCA - 4258, AMAZONAS - 3282, GUAVIARE - 2848, SAN ANDRES - 2413, VICHADA - 1826, GUAINIA - 1191, VAUPES - 872",SEDE NACIONAL - 472760,,,,,,,
CZCreacion,string,31.0,0,0.00%,245,,,,,,,,,,
RegionalDireccionada,string,34.0,0,0.00%,35,"AMAZONAS, ANTIOQUIA, ARAUCA, ATLANTICO, BOGOTA, BOLIVAR, BOYACA, CALDAS, CAQUETA, CASANARE, CAUCA, CESAR, CHOCO, CORDOBA, CUNDINAMARCA, GUAINIA, GUAVIARE, HUILA, INSTITUCIONES AUTORIZADAS ADOPCION, LA GUAJIRA, MAGDALENA, META, NARIÑO, NORTE DE SANTANDER, PUTUMAYO, QUINDIO, RISARALDA, SAN ANDRES, SANTANDER, SEDE NACIONAL, SUCRE, TOLIMA, VALLE DEL CAUCA, VAUPES, VICHADA","BOGOTA - 276818, VALLE DEL CAUCA - 113118, ANTIOQUIA - 89851, CUNDINAMARCA - 69178, ATLANTICO - 55218, TOLIMA - 47158, SANTANDER - 45559, HUILA - 37976, NARIÑO - 35448, CORDOBA - 35248, CALDAS - 34085, BOLIVAR - 31923, META - 31102, RISARALDA - 30394, NORTE DE SANTANDER - 30163, MAGDALENA - 27185, BOYACA - 26573, CAUCA - 24024, CESAR - 21203, QUINDIO - 16821, SUCRE - 15380, LA GUAJIRA - 13419, CASANARE - 13404, CAQUETA - 8616, PUTUMAYO - 7320, ARAUCA - 6053, CHOCO - 5649, AMAZONAS - 3588, GUAVIARE - 3136, SAN ANDRES - 2487, VICHADA - 2000, GUAINIA - 1262, VAUPES - 906, INSTITUCIONES AUTORIZADAS ADOPCION - 873, SEDE NACIONAL - 105",BOGOTA - 276818,,,,,,,


In [0]:
from shutil import copyfile
final = '/dbfs/mnt/data/Data/descriptivas_limpieza_final.xlsx'
temp_file = '/tmp/descriptivas_limpieza_final.xlsx'
descriptivas_limpieza.to_excel(temp_file, engine='openpyxl')

copyfile(temp_file, final)

Out[79]: '/dbfs/mnt/data/Data/descriptivas_limpieza_final.xlsx'

In [0]:
a = 5